In [42]:
beijing.replace(' ', np.nan, inplace=True)
beijing.isna().sum()

date        0
 pm25      38
 pm10       8
 o3         8
 no2       10
 so2       50
 co      2101
dtype: int64

In [36]:
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# PM2.5 예측 모델 (Random Forest)
X_pm25 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm25 = corr_beijing_seoul[' pm25_seoul']

model_pm25_rf = RandomForestRegressor(random_state=42)
model_pm25_rf.fit(X_pm25, y_pm25)

# PM10 예측 모델 (Random Forest)
X_pm10 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm10 = corr_beijing_seoul[' pm10_seoul']

model_pm10_rf = RandomForestRegressor(random_state=42)
model_pm10_rf.fit(X_pm10, y_pm10)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    ' pm25_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    ' pm10_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# PM2.5, PM10 예측
future_X_pm25 = future_beijing[[' pm25_beijing', ' pm10_beijing']]
future_X_pm10 = future_beijing[[' pm25_beijing', ' pm10_beijing']]

future_pred_pm25_rf = model_pm25_rf.predict(future_X_pm25)
future_pred_pm10_rf = model_pm10_rf.predict(future_X_pm10)

future_beijing['Predicted_PM25_RF'] = future_pred_pm25_rf
future_beijing['Predicted_PM10_RF'] = future_pred_pm10_rf

# AQI를 μg/m³로 변환하는 함수 정의 (한국 기준)
def aqi_conversion(pm_value, pm_type):
    if pm_type == 'PM2.5':
        if pm_value <= 12:
            return pm_value * (15.0 / 12)
        elif pm_value <= 35.4:
            return 15.0 + (pm_value - 12) * ((35.0 - 15.0) / (35.4 - 12))
        elif pm_value <= 75.4:
            return 35.0 + (pm_value - 35.4) * ((75.0 - 35.0) / (75.4 - 35.4))
        elif pm_value <= 115.4:
            return 75.0 + (pm_value - 75.4) * ((115.0 - 75.0) / (115.4 - 75.4))
        elif pm_value <= 150.4:
            return 115.0 + (pm_value - 115.4) * ((150.0 - 115.0) / (150.4 - 115.4))
        elif pm_value <= 500:
            return 150.0 + (pm_value - 150.4) * ((350.0 - 150.0) / (500 - 150.4))
        else:
            return 500.0
    elif pm_type == 'PM10':
        if pm_value <= 30:
            return pm_value * (30.0 / 30)
        elif pm_value <= 80:
            return 30.0 + (pm_value - 30) * ((80.0 - 30.0) / (80 - 30))
        elif pm_value <= 150:
            return 80.0 + (pm_value - 80) * ((150.0 - 80.0) / (150 - 80))
        elif pm_value <= 250:
            return 150.0 + (pm_value - 150) * ((250.0 - 150.0) / (250 - 150))
        elif pm_value <= 350:
            return 250.0 + (pm_value - 250) * ((350.0 - 250.0) / (350 - 250))
        elif pm_value <= 420:
            return 350.0 + (pm_value - 350) * ((420.0 - 350.0) / (420 - 350))
        else:
            return 420.0

# 예측 데이터를 AQI로 변환
future_beijing['Predicted_AQI_PM25_RF'] = future_beijing['Predicted_PM25_RF'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
future_beijing['Predicted_AQI_PM10_RF'] = future_beijing['Predicted_PM10_RF'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' pm25', ' pm10']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_PM25', 'Actual_PM10']

# 'Actual_PM25'와 'Actual_PM10' 열을 숫자로 변환
actual_seoul['Actual_PM25'] = pd.to_numeric(actual_seoul['Actual_PM25'], errors='coerce')
actual_seoul['Actual_PM10'] = pd.to_numeric(actual_seoul['Actual_PM10'], errors='coerce')

# 데이터를 AQI로 변환
actual_seoul['Actual_AQI_PM25'] = actual_seoul['Actual_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
actual_seoul['Actual_AQI_PM10'] = actual_seoul['Actual_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 오차 비율 계산
combined_data_rf = pd.merge(future_beijing, actual_seoul, on='date', how='left')
combined_data_rf['PM25_Error_Rate_RF'] = np.abs((combined_data_rf['Actual_AQI_PM25'] - combined_data_rf['Predicted_AQI_PM25_RF']) / combined_data_rf['Actual_AQI_PM25']) * 100
combined_data_rf['PM10_Error_Rate_RF'] = np.abs((combined_data_rf['Actual_AQI_PM10'] - combined_data_rf['Predicted_AQI_PM10_RF']) / combined_data_rf['Actual_AQI_PM10']) * 100

# 함수 정의: AQI 값을 입력받아 범주를 반환하는 함수
def categorize_aqi(aqi_value):
    if aqi_value <= 100:
        return 1
    elif aqi_value <= 200:
        return 2
    elif aqi_value <= 300:
        return 3
    else:
        return 4

# 예측된 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_predicted_aqi_rf(df):
    df['Predicted_AQI_PM25_Category_RF'] = df['Predicted_AQI_PM25_RF'].apply(categorize_aqi)
    df['Predicted_AQI_PM10_Category_RF'] = df['Predicted_AQI_PM10_RF'].apply(categorize_aqi)
    return df

# 실제 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_actual_aqi(df):
    df['Actual_AQI_PM25_Category'] = df['Actual_AQI_PM25'].apply(categorize_aqi)
    df['Actual_AQI_PM10_Category'] = df['Actual_AQI_PM10'].apply(categorize_aqi)
    return df

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories_rf(row):
    if row['Predicted_AQI_PM25_Category_RF'] != row['Actual_AQI_PM25_Category']:
        row['PM25_Category_Difference_RF'] = 1
    else:
        row['PM25_Category_Difference_RF'] = 0
    
    if row['Predicted_AQI_PM10_Category_RF'] != row['Actual_AQI_PM10_Category']:
        row['PM10_Category_Difference_RF'] = 1
    else:
        row['PM10_Category_Difference_RF'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_categorized_rf = categorize_predicted_aqi_rf(combined_data_rf)
combined_data_categorized_rf = categorize_actual_aqi(combined_data_categorized_rf)

# combined_data_categorized에 함수 적용
combined_data_categorized_diff_rf = combined_data_categorized_rf.apply(compare_aqi_categories_rf, axis=1)

# 결과 출력
print("2024년 미세먼지 예측 적중률 (Random Forest)")
# combined_data_categorized_diff_rf[['date', 'Predicted_AQI_PM25_Category_RF', 'Actual_AQI_PM25_Category', 
#                                       'PM25_Category_Difference_RF', 'Predicted_AQI_PM10_Category_RF', 
#                                       'Actual_AQI_PM10_Category', 'PM10_Category_Difference_RF']]

# 2024년 한 해 동안의 데이터 필터링
total_data_rf = combined_data_categorized_diff_rf[(combined_data_categorized_diff_rf['date'] >= '2024-01-01') & 
                                                  (combined_data_categorized_diff_rf['date'] <= '2024-12-31')]

year_data_rf = total_data_rf[(combined_data_categorized_diff_rf['date'] >= '2024-01-01') & 
                                                  (combined_data_categorized_diff_rf['date'] <= '2024-06-30')]

# PM2.5 및 PM10의 카테고리가 같은 비율 계산
total_days_rf = len(year_data_rf)
same_pm25_category_count_rf = year_data_rf[year_data_rf['PM25_Category_Difference_RF'] == 0].shape[0]
same_pm10_category_count_rf = year_data_rf[year_data_rf['PM10_Category_Difference_RF'] == 0].shape[0]

percent_same_pm25_category_rf = (same_pm25_category_count_rf / total_days_rf) * 100
percent_same_pm10_category_rf = (same_pm10_category_count_rf / total_days_rf) * 100

# 결과 출력
print(f"2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율 (Random Forest): {percent_same_pm25_category_rf:.2f}%")
print(f"2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율 (Random Forest): {percent_same_pm10_category_rf:.2f}%\n")

# 특정 날짜의 데이터 출력 함수
def print_aqi_for_date_rf(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data_rf[combined_data_rf['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터 (Random Forest)")
        print(f"예상 PM25 : {round(row['Predicted_AQI_PM25_RF'].values[0])} (Category: {row['Predicted_AQI_PM25_Category_RF'].values[0]})")
        print(f"예상 PM10 : {round(row['Predicted_AQI_PM10_RF'].values[0])} (Category: {row['Predicted_AQI_PM10_Category_RF'].values[0]})\n")

# 특정 날짜의 AQI 데이터를 출력
target_date_rf = '2024-12-30'
print_aqi_for_date_rf(target_date_rf)

from sklearn.metrics import r2_score, mean_squared_error

# PM2.5 예측값 계산
y_pred_pm25 = model_pm25_rf.predict(X_pm25)
# PM10 예측값 계산
y_pred_pm10 = model_pm10_rf.predict(X_pm10)

# R-squared 계산
r2_pm25_rf = r2_score(y_pm25, y_pred_pm25)
r2_pm10_rf = r2_score(y_pm10, y_pred_pm10)

print(f"PM25 예측 모델의 결정 계수 (Random Forest): {r2_pm25_rf:.4f}")
print(f"PM10 예측 모델의 결정 계수 (Random Forest): {r2_pm10_rf:.4f}")

# RMSE 계산
rmse_pm25_rf = mean_squared_error(y_pm25, y_pred_pm25, squared=False)
rmse_pm10_rf = mean_squared_error(y_pm10, y_pred_pm10, squared=False)

print(f"PM25 예측 모델의 RMSE (Random Forest): {rmse_pm25_rf:.4f}")
print(f"PM10 예측 모델의 RMSE (Random Forest): {rmse_pm10_rf:.4f}\n")

# 예측된 AQI 카테고리 계산 함수
def categorize_predicted_aqi_category(df):
    df['Predicted_AQI_PM25_Category'] = df['Predicted_AQI_PM25'].apply(categorize_aqi)
    df['Predicted_AQI_PM10_Category'] = df['Predicted_AQI_PM10'].apply(categorize_aqi)
    return df


# 실제 AQI 카테고리 계산 함수
def categorize_actual_aqi_category(df):
    df['Actual_AQI_PM25_Category'] = df['Actual_AQI_PM25'].apply(categorize_aqi)
    df['Actual_AQI_PM10_Category'] = df['Actual_AQI_PM10'].apply(categorize_aqi)
    return df

# 카테고리화된 데이터를 기반으로 RMSE 계산
rmse_pm25_category = mean_squared_error(combined_data_categorized_rf['Actual_AQI_PM25_Category'], 
                                        combined_data_categorized_rf['Predicted_AQI_PM25_Category_RF'], squared=False)

rmse_pm10_category = mean_squared_error(combined_data_categorized_rf['Actual_AQI_PM10_Category'], 
                                        combined_data_categorized_rf['Predicted_AQI_PM10_Category_RF'], squared=False)



# 카테고리화된 데이터를 기반으로 결정 계수 계산
r2_pm25_category = accuracy_score(combined_data_categorized_rf['Actual_AQI_PM25_Category'], 
                                  combined_data_categorized_rf['Predicted_AQI_PM25_Category_RF'])

r2_pm10_category = accuracy_score(combined_data_categorized_rf['Actual_AQI_PM10_Category'], 
                                  combined_data_categorized_rf['Predicted_AQI_PM10_Category_RF'])

print(f"PM25 예측 모델의 카테고리별 결정 계수 (Random Forest) : {r2_pm25_category:.4f}")
print(f"PM10 예측 모델의 카테고리별 결정 계수 (Random Forest) : {r2_pm10_category:.4f}")
print(f"PM25 예측 모델의 카테고리별 RMSE (Random Forest) : {rmse_pm25_category:.4f}")
print(f"PM10 예측 모델의 카테고리별 RMSE (Random Forest) : {rmse_pm10_category:.4f}")



2024년 미세먼지 예측 적중률 (Random Forest)
2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율 (Random Forest): 65.93%
2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율 (Random Forest): 98.90%

2024-12-30 대기질 예측 데이터 (Random Forest)
예상 PM25 : 94 (Category: 1)
예상 PM10 : 48 (Category: 1)

PM25 예측 모델의 결정 계수 (Random Forest): 0.5876
PM10 예측 모델의 결정 계수 (Random Forest): 0.5239
PM25 예측 모델의 RMSE (Random Forest): 22.0030
PM10 예측 모델의 RMSE (Random Forest): 16.1129

PM25 예측 모델의 카테고리별 결정 계수 (Random Forest) : 0.3443
PM10 예측 모델의 카테고리별 결정 계수 (Random Forest) : 0.5082
PM25 예측 모델의 카테고리별 RMSE (Random Forest) : 2.0319
PM10 예측 모델의 카테고리별 RMSE (Random Forest) : 2.0934


In [24]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import accuracy_score
# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# PM2.5 예측 모델
X_pm25 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm25 = corr_beijing_seoul[' pm25_seoul']

# PM10 예측 모델
X_pm10 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm10 = corr_beijing_seoul[' pm10_seoul']

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# PM2.5, PM10 데이터 준비
X_pm25, y_pm25 = prepare_data(X_pm25, y_pm25)
X_pm10, y_pm10 = prepare_data(X_pm10, y_pm10)

# 모델 생성 및 학습
model_pm25 = LinearRegression()
model_pm25.fit(X_pm25, y_pm25)

model_pm10 = LinearRegression()
model_pm10.fit(X_pm10, y_pm10)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    ' pm25_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    ' pm10_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# PM2.5, PM10 예측
future_X_pm25 = future_beijing[[' pm25_beijing', ' pm10_beijing']]
future_X_pm10 = future_beijing[[' pm25_beijing', ' pm10_beijing']]

future_pred_pm25 = model_pm25.predict(future_X_pm25)
future_pred_pm10 = model_pm10.predict(future_X_pm10)

future_beijing['Predicted_PM25'] = future_pred_pm25
future_beijing['Predicted_PM10'] = future_pred_pm10

# AQI를 μg/m³로 변환하는 함수 정의 (한국 기준)
def aqi_conversion(pm_value, pm_type):
    if pm_type == 'PM2.5':
        if pm_value <= 12:
            return pm_value * (15.0 / 12)
        elif pm_value <= 35.4:
            return 15.0 + (pm_value - 12) * ((35.0 - 15.0) / (35.4 - 12))
        elif pm_value <= 75.4:
            return 35.0 + (pm_value - 35.4) * ((75.0 - 35.0) / (75.4 - 35.4))
        elif pm_value <= 115.4:
            return 75.0 + (pm_value - 75.4) * ((115.0 - 75.0) / (115.4 - 75.4))
        elif pm_value <= 150.4:
            return 115.0 + (pm_value - 115.4) * ((150.0 - 115.0) / (150.4 - 115.4))
        elif pm_value <= 500:
            return 150.0 + (pm_value - 150.4) * ((350.0 - 150.0) / (500 - 150.4))
        else:
            return 500.0
    elif pm_type == 'PM10':
        if pm_value <= 30:
            return pm_value * (30.0 / 30)
        elif pm_value <= 80:
            return 30.0 + (pm_value - 30) * ((80.0 - 30.0) / (80 - 30))
        elif pm_value <= 150:
            return 80.0 + (pm_value - 80) * ((150.0 - 80.0) / (150 - 80))
        elif pm_value <= 250:
            return 150.0 + (pm_value - 150) * ((250.0 - 150.0) / (250 - 150))
        elif pm_value <= 350:
            return 250.0 + (pm_value - 250) * ((350.0 - 250.0) / (350 - 250))
        elif pm_value <= 420:
            return 350.0 + (pm_value - 350) * ((420.0 - 350.0) / (420 - 350))
        else:
            return 420.0

# 예측 데이터를 AQI로 변환
future_beijing['Predicted_AQI_PM25'] = future_beijing['Predicted_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
future_beijing['Predicted_AQI_PM10'] = future_beijing['Predicted_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' pm25', ' pm10']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_PM25', 'Actual_PM10']

# 'Actual_PM25'와 'Actual_PM10' 열을 숫자로 변환
actual_seoul['Actual_PM25'] = pd.to_numeric(actual_seoul['Actual_PM25'], errors='coerce')
actual_seoul['Actual_PM10'] = pd.to_numeric(actual_seoul['Actual_PM10'], errors='coerce')

# 데이터를 AQI로 변환
actual_seoul['Actual_AQI_PM25'] = actual_seoul['Actual_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
actual_seoul['Actual_AQI_PM10'] = actual_seoul['Actual_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 오차 비율 계산
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')
combined_data['PM25_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM25'] - combined_data['Predicted_AQI_PM25']) / combined_data['Actual_AQI_PM25']) * 100
combined_data['PM10_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM10'] - combined_data['Predicted_AQI_PM10']) / combined_data['Actual_AQI_PM10']) * 100

# 함수 정의: AQI 값을 입력받아 범주를 반환하는 함수
def categorize_aqi(aqi_value):
    if aqi_value <= 100:
        return 1
    elif aqi_value <= 200:
        return 2
    elif aqi_value <= 300:
        return 3
    else:
        return 4

# 예측된 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_predicted_aqi(df):
    df['Predicted_AQI_PM25_Category'] = df['Predicted_AQI_PM25'].apply(categorize_aqi)
    df['Predicted_AQI_PM10_Category'] = df['Predicted_AQI_PM10'].apply(categorize_aqi)
    return df

# 실제 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_actual_aqi(df):
    df['Actual_AQI_PM25_Category'] = df['Actual_AQI_PM25'].apply(categorize_aqi)
    df['Actual_AQI_PM10_Category'] = df['Actual_AQI_PM10'].apply(categorize_aqi)
    return df

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories(row):
    if row['Predicted_AQI_PM25_Category'] != row['Actual_AQI_PM25_Category']:
        row['PM25_Category_Difference'] = 1
    else:
        row['PM25_Category_Difference'] = 0
    
    if row['Predicted_AQI_PM10_Category'] != row['Actual_AQI_PM10_Category']:
        row['PM10_Category_Difference'] = 1
    else:
        row['PM10_Category_Difference'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_categorized = categorize_predicted_aqi(combined_data)
combined_data_categorized = categorize_actual_aqi(combined_data_categorized)

# combined_data_categorized에 함수 적용
combined_data_categorized_diff = combined_data_categorized.apply(compare_aqi_categories, axis=1)

# 결과 출력
print("2024년 미세먼지 예측 적중률")
# combined_data_categorized_diff[['date', 'Predicted_AQI_PM25_Category', 'Actual_AQI_PM25_Category', 
#                                       'PM25_Category_Difference', 'Predicted_AQI_PM10_Category', 
#                                       'Actual_AQI_PM10_Category', 'PM10_Category_Difference']]

# 2024년 한 해 동안의 데이터 필터링
total_data = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-12-31')]

year_data = total_data[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-06-30')]

# PM2.5 및 PM10의 카테고리가 같은 비율 계산
total_days = len(year_data)
same_pm25_category_count = year_data[year_data['PM25_Category_Difference'] == 0].shape[0]
same_pm10_category_count = year_data[year_data['PM10_Category_Difference'] == 0].shape[0]

percent_same_pm25_category = (same_pm25_category_count / total_days) * 100
percent_same_pm10_category = (same_pm10_category_count / total_days) * 100

# 결과 출력
print(f"2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: {percent_same_pm25_category:.2f}%")
print(f"2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: {percent_same_pm10_category:.2f}%\n")

# 특정 날짜의 데이터 출력 함수
def print_aqi_for_date(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data[combined_data['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터")
        print(f"예상 PM25 : {round(row['Predicted_AQI_PM25'].values[0])} (Category: {row['Predicted_AQI_PM25_Category'].values[0]})")
        print(f"예상 PM10 : {round(row['Predicted_AQI_PM10'].values[0])} (Category: {row['Predicted_AQI_PM10_Category'].values[0]})\n")

# 특정 날짜의 AQI 데이터를 출력
target_date = '2024-12-20'
print_aqi_for_date(target_date)


# 예측값 계산
y_pred_pm25 = model_pm25.predict(X_pm25)
y_pred_pm10 = model_pm10.predict(X_pm10)

# R-squared 계산
r2_pm25 = r2_score(y_pm25, y_pred_pm25)
r2_pm10 = r2_score(y_pm10, y_pred_pm10)

print(f"PM25 예측 모델의 결정 계수 (R-squared): {r2_pm25:.4f}")
print(f"PM10 예측 모델의 결정 계수 (R-squared): {r2_pm10:.4f}")

# RMSE 계산
rmse_pm25 = mean_squared_error(y_pm25, y_pred_pm25, squared=False)
rmse_pm10 = mean_squared_error(y_pm10, y_pred_pm10, squared=False)

print(f"PM2.5 예측 모델의 RMSE: {rmse_pm25:.4f}")
print(f"PM10 예측 모델의 RMSE: {rmse_pm10:.4f}\n")


# 예측된 AQI 카테고리 계산 함수
def categorize_predicted_aqi_category(df):
    df['Predicted_AQI_PM25_Category'] = df['Predicted_AQI_PM25'].apply(categorize_aqi)
    df['Predicted_AQI_PM10_Category'] = df['Predicted_AQI_PM10'].apply(categorize_aqi)
    return df

# 카테고리화된 데이터 준비
combined_data_categorized = categorize_predicted_aqi_category(combined_data)

# 실제 AQI 카테고리 계산 함수
def categorize_actual_aqi_category(df):
    df['Actual_AQI_PM25_Category'] = df['Actual_AQI_PM25'].apply(categorize_aqi)
    df['Actual_AQI_PM10_Category'] = df['Actual_AQI_PM10'].apply(categorize_aqi)
    return df

# 카테고리화된 실제 데이터 준비
combined_data_categorized = categorize_actual_aqi_category(combined_data_categorized)

# 카테고리화된 데이터를 기반으로 RMSE 계산
rmse_pm25_category = mean_squared_error(combined_data_categorized['Actual_AQI_PM25_Category'], 
                                        combined_data_categorized['Predicted_AQI_PM25_Category'], squared=False)

rmse_pm10_category = mean_squared_error(combined_data_categorized['Actual_AQI_PM10_Category'], 
                                        combined_data_categorized['Predicted_AQI_PM10_Category'], squared=False)

print(f"PM2.5 예측 모델의 카테고리별 RMSE: {rmse_pm25_category:.4f}")
print(f"PM10 예측 모델의 카테고리별 RMSE: {rmse_pm10_category:.4f}")

# 카테고리화된 데이터를 기반으로 결정 계수 계산
# 실제 데이터가 있는 경우만 필터링
filtered_data = combined_data_categorized[combined_data_categorized['Actual_AQI_PM25_Category'] != 4]

# PM2.5 예측 모델의 카테고리별 결정 계수 계산
r2_pm25_category_filtered = accuracy_score(filtered_data['Actual_AQI_PM25_Category'], 
                                          filtered_data['Predicted_AQI_PM25_Category'])

# PM10 예측 모델의 카테고리별 결정 계수 계산
r2_pm10_category_filtered = accuracy_score(filtered_data['Actual_AQI_PM10_Category'], 
                                          filtered_data['Predicted_AQI_PM10_Category'])

print(f"PM25 예측 모델의 카테고리별 결정 계수 (필터링 후): {r2_pm25_category_filtered:.4f}")
print(f"PM10 예측 모델의 카테고리별 결정 계수 (필터링 후): {r2_pm10_category_filtered:.4f}")

2024년 미세먼지 예측 적중률
2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: 79.67%
2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: 98.90%

2024-12-20 대기질 예측 데이터
예상 PM25 : 91 (Category: 1)
예상 PM10 : 51 (Category: 1)

PM25 예측 모델의 결정 계수 (R-squared): 0.0393
PM10 예측 모델의 결정 계수 (R-squared): 0.0562
PM2.5 예측 모델의 RMSE: 34.2497
PM10 예측 모델의 RMSE: 21.5085

PM2.5 예측 모델의 카테고리별 RMSE: 2.1110
PM10 예측 모델의 카테고리별 RMSE: 2.0934
PM25 예측 모델의 카테고리별 결정 계수 (필터링 후): 0.7989
PM10 예측 모델의 카테고리별 결정 계수 (필터링 후): 0.9841


In [21]:
len(y_pred_pm25)

1912

In [11]:
import numpy as np
from sklearn.metrics import accuracy_score

# 실제 데이터가 있는 경우만 필터링
filtered_data = combined_data_categorized[combined_data_categorized['Actual_AQI_PM25_Category'] != 4]

# PM2.5 예측 모델의 카테고리별 결정 계수 계산
r2_pm25_category_filtered = accuracy_score(filtered_data['Actual_AQI_PM25_Category'], 
                                          filtered_data['Predicted_AQI_PM25_Category'])

# PM10 예측 모델의 카테고리별 결정 계수 계산
r2_pm10_category_filtered = accuracy_score(filtered_data['Actual_AQI_PM10_Category'], 
                                          filtered_data['Predicted_AQI_PM10_Category'])

print(f"PM25 예측 모델의 카테고리별 결정 계수 (필터링 후): {r2_pm25_category_filtered:.4f}")
print(f"PM10 예측 모델의 카테고리별 결정 계수 (필터링 후): {r2_pm10_category_filtered:.4f}")

PM2.5 예측 모델의 카테고리별 결정 계수 (필터링 후): 0.7989
PM10 예측 모델의 카테고리별 결정 계수 (필터링 후): 0.9841


In [35]:
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

corr_beijing_seoul.fillna(corr_beijing_seoul.mean(), inplace=True)
corr_beijing_seoul.fillna(0, inplace=True)

# O3 예측 모델 (Random Forest)
X_o3 = corr_beijing_seoul[[' o3_beijing', ' co_beijing']]
y_o3 = corr_beijing_seoul[' o3_seoul']

model_o3_rf = RandomForestRegressor(random_state=42)
model_o3_rf.fit(X_o3, y_o3)

# CO 예측 모델 (Random Forest)
X_co = corr_beijing_seoul[[' o3_beijing', ' co_beijing']]
y_co = corr_beijing_seoul[' co_seoul']

model_co_rf = RandomForestRegressor(random_state=42)
model_co_rf.fit(X_co, y_co)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    ' o3_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    ' co_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# O3, CO 예측
future_X_o3 = future_beijing[[' o3_beijing', ' co_beijing']]
future_X_co = future_beijing[[' o3_beijing', ' co_beijing']]

future_pred_o3_rf = model_o3_rf.predict(future_X_o3)
future_pred_co_rf = model_co_rf.predict(future_X_co)

future_beijing['Predicted_O3_RF'] = future_pred_o3_rf
future_beijing['Predicted_CO_RF'] = future_pred_co_rf

# AQI를 μg/m³로 변환하는 함수 정의 (한국 기준)
def aqi_conversion(pm_value, pm_type):
    if pm_type == 'O3':
        if pm_value <= 0.03:
            return pm_value * (50.0 / 0.03)
        elif pm_value <= 0.09:
            return 50.0 + (pm_value - 0.03) * ((100.0 - 50.0) / (0.09 - 0.03))
        elif pm_value <= 0.15:
            return 100.0 + (pm_value - 0.09) * ((150.0 - 100.0) / (0.15 - 0.09))
        elif pm_value <= 0.20:
            return 150.0 + (pm_value - 0.15) * ((200.0 - 150.0) / (0.20 - 0.15))
        elif pm_value <= 0.60:
            return 200.0 + (pm_value - 0.20) * ((300.0 - 200.0) / (0.60 - 0.20))
        else:
            return 0
    elif pm_type == 'CO':
        if pm_value <= 2.0:
            return pm_value * (50.0 / 2.0)
        elif pm_value <= 9.0:
            return 50.0 + (pm_value - 2.0) * ((100.0 - 50.0) / (9.0 - 2.0))
        elif pm_value <= 15.0:
            return 100.0 + (pm_value - 9.0) * ((150.0 - 100.0) / (15.0 - 9.0))
        elif pm_value <= 30.0:
            return 150.0 + (pm_value - 15.0) * ((200.0 - 150.0) / (30.0 - 15.0))
        elif pm_value <= 40.0:
            return 200.0 + (pm_value - 30.0) * ((300.0 - 200.0) / (40.0 - 30.0))
        else:
            return 0

# 예측 데이터를 AQI로 변환
future_beijing['Predicted_AQI_O3_RF'] = future_beijing['Predicted_O3_RF'].apply(lambda x: aqi_conversion(x, 'O3'))
future_beijing['Predicted_AQI_CO_RF'] = future_beijing['Predicted_CO_RF'].apply(lambda x: aqi_conversion(x, 'CO'))

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' o3', ' co']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_O3', 'Actual_CO']

# 'Actual_O3'와 'Actual_CO' 열을 숫자로 변환
actual_seoul['Actual_O3'] = pd.to_numeric(actual_seoul['Actual_O3'], errors='coerce')
actual_seoul['Actual_CO'] = pd.to_numeric(actual_seoul['Actual_CO'], errors='coerce')

# 데이터를 AQI로 변환
actual_seoul['Actual_AQI_O3'] = actual_seoul['Actual_O3'].apply(lambda x: aqi_conversion(x, 'O3'))
actual_seoul['Actual_AQI_CO'] = actual_seoul['Actual_CO'].apply(lambda x: aqi_conversion(x, 'CO'))

# 오차 비율 계산
combined_data_rf = pd.merge(future_beijing, actual_seoul, on='date', how='left')
combined_data_rf['O3_Error_Rate_RF'] = np.abs((combined_data_rf['Actual_AQI_O3'] - combined_data_rf['Predicted_AQI_O3_RF']) / combined_data_rf['Actual_AQI_O3']) * 100
combined_data_rf['CO_Error_Rate_RF'] = np.abs((combined_data_rf['Actual_AQI_CO'] - combined_data_rf['Predicted_AQI_CO_RF']) / combined_data_rf['Actual_AQI_CO']) * 100

# 함수 정의: AQI 값을 입력받아 범주를 반환하는 함수
def categorize_aqi(aqi_value):
    if aqi_value <= 100:
        return 1
    elif aqi_value <= 200:
        return 2
    elif aqi_value <= 300:
        return 3
    else:
        return 4

# 예측된 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_predicted_aqi_rf(df):
    df['Predicted_AQI_O3_Category_RF'] = df['Predicted_AQI_O3_RF'].apply(categorize_aqi)
    df['Predicted_AQI_CO_Category_RF'] = df['Predicted_AQI_CO_RF'].apply(categorize_aqi)
    return df

# 실제 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_actual_aqi(df):
    df['Actual_AQI_O3_Category'] = df['Actual_AQI_O3'].apply(categorize_aqi)
    df['Actual_AQI_CO_Category'] = df['Actual_AQI_CO'].apply(categorize_aqi)
    return df

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories_rf(row):
    if row['Predicted_AQI_O3_Category_RF'] != row['Actual_AQI_O3_Category']:
        row['O3_Category_Difference_RF'] = 1
    else:
        row['O3_Category_Difference_RF'] = 0
    
    if row['Predicted_AQI_CO_Category_RF'] != row['Actual_AQI_CO_Category']:
        row['CO_Category_Difference_RF'] = 1
    else:
        row['CO_Category_Difference_RF'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_categorized_rf = categorize_predicted_aqi_rf(combined_data_rf)
combined_data_categorized_rf = categorize_actual_aqi(combined_data_categorized_rf)

# combined_data_categorized에 함수 적용
combined_data_categorized_diff_rf = combined_data_categorized_rf.apply(compare_aqi_categories_rf, axis=1)

# 결과 출력
print("2024년 오존 및 일산화탄소 예측 적중률 (Random Forest)")

# 2024년 한 해 동안의 데이터 필터링
total_data_rf = combined_data_categorized_diff_rf[(combined_data_categorized_diff_rf['date'] >= '2024-01-01') & 
                                                  (combined_data_categorized_diff_rf['date'] <= '2024-12-31')]

year_data_rf = total_data_rf[(combined_data_categorized_diff_rf['date'] >= '2024-01-01') & 
                                                  (combined_data_categorized_diff_rf['date'] <= '2024-06-30')]

# PM2.5 및 PM10의 카테고리가 같은 비율 계산
total_days_rf = len(total_data_rf)
same_o3_category_count_rf = year_data_rf[year_data_rf['O3_Category_Difference_RF'] == 0].shape[0]
same_co_category_count_rf = year_data_rf[year_data_rf['CO_Category_Difference_RF'] == 0].shape[0]
percent_same_o3_category_rf = (same_o3_category_count_rf / total_days_rf) * 100
percent_same_co_category_rf = (same_co_category_count_rf / total_days_rf) * 100

# 결과 출력
print(f"2024년 동안 오존의 예측 카테고리가 실제와 같은 비율 (Random Forest): {percent_same_o3_category_rf:.2f}%")
print(f"2024년 동안 일산화탄소의 예측 카테고리가 실제와 같은 비율 (Random Forest): {percent_same_co_category_rf:.2f}%\n")

# 특정 날짜의 데이터 출력 함수
def print_aqi_for_date_rf(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data_rf[combined_data_rf['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터 (Random Forest)")
        print(f"예상 O3 : {round(row['Predicted_AQI_O3_RF'].values[0])} (Category: {row['Predicted_AQI_O3_Category_RF'].values[0]})")
        print(f"예상 CO : {round(row['Predicted_AQI_CO_RF'].values[0])} (Category: {row['Predicted_AQI_CO_Category_RF'].values[0]})\n")

# 특정 날짜의 AQI 데이터를 출력
target_date_rf = '2024-12-30'
print_aqi_for_date_rf(target_date_rf)

from sklearn.metrics import r2_score, mean_squared_error

# O3 예측값 계산
y_pred_o3 = model_o3_rf.predict(X_o3)
# CO 예측값 계산
y_pred_co = model_co_rf.predict(X_co)

# R-squared 계산
r2_o3_rf = r2_score(y_o3, y_pred_o3)
r2_co_rf = r2_score(y_co, y_pred_co)

print(f"O3 예측 모델의 결정 계수 (Random Forest): {r2_o3_rf:.4f}")
print(f"CO 예측 모델의 결정 계수 (Random Forest): {r2_co_rf:.4f}")

# RMSE 계산
rmse_o3_rf = mean_squared_error(y_o3, y_pred_o3, squared=False)
rmse_co_rf = mean_squared_error(y_co, y_pred_co, squared=False)

print(f"O3 예측 모델의 RMSE (Random Forest): {rmse_o3_rf:.4f}")
print(f"CO 예측 모델의 RMSE (Random Forest): {rmse_co_rf:.4f}\n")

# 예측된 AQI 카테고리 계산 함수
def categorize_predicted_aqi_category(df):
    df['Predicted_AQI_O3_Category'] = df['Predicted_AQI_O3'].apply(categorize_aqi)
    df['Predicted_AQI_CO_Category'] = df['Predicted_AQI_CO'].apply(categorize_aqi)
    return df


# 실제 AQI 카테고리 계산 함수
def categorize_actual_aqi_category(df):
    df['Actual_AQI_O3_Category'] = df['Actual_AQI_O3'].apply(categorize_aqi)
    df['Actual_AQI_CO_Category'] = df['Actual_AQI_CO'].apply(categorize_aqi)
    return df

# 카테고리화된 데이터를 기반으로 RMSE 계산
rmse_o3_category = mean_squared_error(combined_data_categorized_rf['Actual_AQI_O3_Category'], 
                                      combined_data_categorized_rf['Predicted_AQI_O3_Category_RF'], squared=False)

rmse_co_category = mean_squared_error(combined_data_categorized_rf['Actual_AQI_CO_Category'], 
                                      combined_data_categorized_rf['Predicted_AQI_CO_Category_RF'], squared=False)

# 카테고리화된 데이터를 기반으로 결정 계수 계산
r2_o3_category = accuracy_score(combined_data_categorized_rf['Actual_AQI_O3_Category'], 
                                combined_data_categorized_rf['Predicted_AQI_O3_Category_RF'])

r2_co_category = accuracy_score(combined_data_categorized_rf['Actual_AQI_CO_Category'], 
                                combined_data_categorized_rf['Predicted_AQI_CO_Category_RF'])

print(f"O3 예측 모델의 카테고리별 결정 계수 (Random Forest) : {r2_o3_category:.4f}")
print(f"CO 예측 모델의 카테고리별 결정 계수 (Random Forest) : {r2_co_category:.4f}")
print(f"O3 예측 모델의 카테고리별 RMSE (Random Forest) : {rmse_o3_category:.4f}")
print(f"CO 예측 모델의 카테고리별 RMSE (Random Forest) : {rmse_co_category:.4f}")


2024년 오존 및 일산화탄소 예측 적중률 (Random Forest)
2024년 동안 오존의 예측 카테고리가 실제와 같은 비율 (Random Forest): 49.73%
2024년 동안 일산화탄소의 예측 카테고리가 실제와 같은 비율 (Random Forest): 48.36%

2024-12-30 대기질 예측 데이터 (Random Forest)
예상 O3 : 0 (Category: 1)
예상 CO : 65 (Category: 1)

O3 예측 모델의 결정 계수 (Random Forest): 0.0843
CO 예측 모델의 결정 계수 (Random Forest): 0.0342
O3 예측 모델의 RMSE (Random Forest): 16.1388
CO 예측 모델의 RMSE (Random Forest): 2.1166

O3 예측 모델의 카테고리별 결정 계수 (Random Forest) : 0.5164
CO 예측 모델의 카테고리별 결정 계수 (Random Forest) : 0.5027
O3 예측 모델의 카테고리별 RMSE (Random Forest) : 2.0863
CO 예측 모델의 카테고리별 RMSE (Random Forest) : 2.0895


In [37]:
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# corr_beijing_seoul.fillna(corr_beijing_seoul.mean(), inplace=True)
# corr_beijing_seoul.fillna(0, inplace=True)

# NO2 예측 모델 (Random Forest)
X_no2 = corr_beijing_seoul[[' no2_beijing', ' so2_beijing']]
y_no2 = corr_beijing_seoul[' no2_seoul']

model_no2_rf = RandomForestRegressor(random_state=100)
model_no2_rf.fit(X_no2, y_no2)

# SO2 예측 모델 (Random Forest)
X_so2 = corr_beijing_seoul[[' no2_beijing', ' so2_beijing']]
y_so2 = corr_beijing_seoul[' so2_seoul']

model_so2_rf = RandomForestRegressor(random_state=100)
model_so2_rf.fit(X_so2, y_so2)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    ' no2_beijing': [random.randint(10, 100) for _ in range(len(future_dates))],
    ' so2_beijing': [random.randint(1, 20) for _ in range(len(future_dates))]
})

# NO2, SO2 예측
future_X_no2 = future_beijing[[' no2_beijing', ' so2_beijing']]
future_X_so2 = future_beijing[[' no2_beijing', ' so2_beijing']]

future_pred_no2_rf = model_no2_rf.predict(future_X_no2)
future_pred_so2_rf = model_so2_rf.predict(future_X_so2)

future_beijing['Predicted_NO2_RF'] = future_pred_no2_rf
future_beijing['Predicted_SO2_RF'] = future_pred_so2_rf

# AQI를 μg/m³로 변환하는 함수 정의 (한국 기준)
def aqi_conversion(pm_value, pm_type):
    if pm_type == 'NO2':
        if pm_value <= 0.03:
            return pm_value * (50.0 / 0.03)
        elif pm_value <= 0.06:
            return 50.0 + (pm_value - 0.03) * ((100.0 - 50.0) / (0.06 - 0.03))
        elif pm_value <= 0.12:
            return 100.0 + (pm_value - 0.06) * ((150.0 - 100.0) / (0.12 - 0.06))
        elif pm_value <= 0.19:
            return 150.0 + (pm_value - 0.12) * ((200.0 - 150.0) / (0.19 - 0.12))
        elif pm_value <= 1.00:
            return 200.0 + (pm_value - 0.19) * ((300.0 - 200.0) / (1.00 - 0.19))
        else:
            return 0
    elif pm_type == 'SO2':
        if pm_value <= 0.02:
            return pm_value * (50.0 / 0.02)
        elif pm_value <= 0.05:
            return 50.0 + (pm_value - 0.02) * ((100.0 - 50.0) / (0.05 - 0.02))
        elif pm_value <= 0.15:
            return 100.0 + (pm_value - 0.05) * ((150.0 - 100.0) / (0.15 - 0.05))
        elif pm_value <= 0.36:
            return 150.0 + (pm_value - 0.15) * ((200.0 - 150.0) / (0.36 - 0.15))
        elif pm_value <= 0.61:
            return 200.0 + (pm_value - 0.36) * ((300.0 - 200.0) / (0.61 - 0.36))
        else:
            return 0

# 예측 데이터를 AQI로 변환
future_beijing['Predicted_AQI_NO2_RF'] = future_beijing['Predicted_NO2_RF'].apply(lambda x: aqi_conversion(x, 'NO2'))
future_beijing['Predicted_AQI_SO2_RF'] = future_beijing['Predicted_SO2_RF'].apply(lambda x: aqi_conversion(x, 'SO2'))

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' no2', ' so2']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_NO2', 'Actual_SO2']

# 'Actual_NO2'와 'Actual_SO2' 열을 숫자로 변환
actual_seoul['Actual_NO2'] = pd.to_numeric(actual_seoul['Actual_NO2'], errors='coerce')
actual_seoul['Actual_SO2'] = pd.to_numeric(actual_seoul['Actual_SO2'], errors='coerce')

# 데이터를 AQI로 변환
actual_seoul['Actual_AQI_NO2'] = actual_seoul['Actual_NO2'].apply(lambda x: aqi_conversion(x, 'NO2'))
actual_seoul['Actual_AQI_SO2'] = actual_seoul['Actual_SO2'].apply(lambda x: aqi_conversion(x, 'SO2'))

# 오차 비율 계산
combined_data_rf = pd.merge(future_beijing, actual_seoul, on='date', how='left')
combined_data_rf['NO2_Error_Rate_RF'] = np.abs((combined_data_rf['Actual_AQI_NO2'] - combined_data_rf['Predicted_AQI_NO2_RF']) / combined_data_rf['Actual_AQI_NO2']) * 100
combined_data_rf['SO2_Error_Rate_RF'] = np.abs((combined_data_rf['Actual_AQI_SO2'] - combined_data_rf['Predicted_AQI_SO2_RF']) / combined_data_rf['Actual_AQI_SO2']) * 100

# 함수 정의: AQI 값을 입력받아 범주를 반환하는 함수
def categorize_aqi(aqi_value):
    if aqi_value <= 100:
        return 1
    elif aqi_value <= 200:
        return 2
    elif aqi_value <= 300:
        return 3
    else:
        return 4

# 예측된 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_predicted_aqi_rf(df):
    df['Predicted_AQI_NO2_Category_RF'] = df['Predicted_AQI_NO2_RF'].apply(categorize_aqi)
    df['Predicted_AQI_SO2_Category_RF'] = df['Predicted_AQI_SO2_RF'].apply(categorize_aqi)
    return df

# 실제 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_actual_aqi(df):
    df['Actual_AQI_NO2_Category'] = df['Actual_AQI_NO2'].apply(categorize_aqi)
    df['Actual_AQI_SO2_Category'] = df['Actual_AQI_SO2'].apply(categorize_aqi)
    return df

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories_rf(row):
    if row['Predicted_AQI_NO2_Category_RF'] != row['Actual_AQI_NO2_Category']:
        row['NO2_Category_Difference_RF'] = 1
    else:
        row['NO2_Category_Difference_RF'] = 0
    
    if row['Predicted_AQI_SO2_Category_RF'] != row['Actual_AQI_SO2_Category']:
        row['SO2_Category_Difference_RF'] = 1
    else:
        row['SO2_Category_Difference_RF'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_categorized_rf = categorize_predicted_aqi_rf(combined_data_rf)
combined_data_categorized_rf = categorize_actual_aqi(combined_data_categorized_rf)

# combined_data_categorized에 함수 적용
combined_data_categorized_diff_rf = combined_data_categorized_rf.apply(compare_aqi_categories_rf, axis=1)

# 결과 출력
print("2024년 NO2 및 SO2 예측 적중률 (Random Forest)")

# 2024년 한 해 동안의 데이터 필터링
total_data_rf = combined_data_categorized_diff_rf[(combined_data_categorized_diff_rf['date'] >= '2024-01-01') & 
                                                  (combined_data_categorized_diff_rf['date'] <= '2024-12-31')]

# NO2 및 SO2의 카테고리가 같은 비율 계산
total_days_rf = len(total_data_rf)
same_no2_category_count_rf = total_data_rf[total_data_rf['NO2_Category_Difference_RF'] == 0].shape[0]
same_so2_category_count_rf = total_data_rf[total_data_rf['SO2_Category_Difference_RF'] == 0].shape[0]

percent_same_no2_category_rf = (same_no2_category_count_rf / total_days_rf) * 100
percent_same_so2_category_rf = (same_so2_category_count_rf / total_days_rf) * 100

# 결과 출력
print(f"2024년 동안 NO2의 예측 카테고리가 실제와 같은 비율 (Random Forest): {percent_same_no2_category_rf:.2f}%")
print(f"2024년 동안 SO2의 예측 카테고리가 실제와 같은 비율 (Random Forest): {percent_same_so2_category_rf:.2f}%\n")

# 특정 날짜의 데이터 출력 함수
def print_aqi_for_date_rf(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data_rf[combined_data_rf['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터 (Random Forest)")
        print(f"예상 NO2 : {round(row['Predicted_AQI_NO2_RF'].values[0])} (Category: {row['Predicted_AQI_NO2_Category_RF'].values[0]})")
        print(f"예상 SO2 : {round(row['Predicted_AQI_SO2_RF'].values[0])} (Category: {row['Predicted_AQI_SO2_Category_RF'].values[0]})\n")

# 특정 날짜의 AQI 데이터를 출력
target_date_rf = '2024-9-30'
print_aqi_for_date_rf(target_date_rf)

from sklearn.metrics import r2_score, mean_squared_error

# NO2 예측값 계산
y_pred_no2 = model_no2_rf.predict(X_no2)
# SO2 예측값 계산
y_pred_so2 = model_so2_rf.predict(X_so2)

# R-squared 계산
r2_no2_rf = r2_score(y_no2, y_pred_no2)
r2_so2_rf = r2_score(y_so2, y_pred_so2)

print(f"NO2 예측 모델의 결정 계수 (Random Forest): {r2_no2_rf:.4f}")
print(f"SO2 예측 모델의 결정 계수 (Random Forest): {r2_so2_rf:.4f}")

# RMSE 계산
rmse_no2_rf = mean_squared_error(y_no2, y_pred_no2, squared=False)
rmse_so2_rf = mean_squared_error(y_so2, y_pred_so2, squared=False)

print(f"NO2 예측 모델의 RMSE (Random Forest): {rmse_no2_rf:.4f}")
print(f"SO2 예측 모델의 RMSE (Random Forest): {rmse_so2_rf:.4f}\n")

# 예측된 AQI 카테고리 계산 함수
def categorize_predicted_aqi_category(df):
    df['Predicted_AQI_NO2_Category'] = df['Predicted_AQI_NO2'].apply(categorize_aqi)
    df['Predicted_AQI_SO2_Category'] = df['Predicted_AQI_SO2'].apply(categorize_aqi)
    return df


# 실제 AQI 카테고리 계산 함수
def categorize_actual_aqi_category(df):
    df['Actual_AQI_NO2_Category'] = df['Actual_AQI_NO2'].apply(categorize_aqi)
    df['Actual_AQI_SO2_Category'] = df['Actual_AQI_SO2'].apply(categorize_aqi)
    return df

# 카테고리화된 데이터를 기반으로 RMSE 계산
rmse_no2_category = mean_squared_error(combined_data_categorized_rf['Actual_AQI_NO2_Category'], 
                                      combined_data_categorized_rf['Predicted_AQI_NO2_Category_RF'], squared=False)

rmse_so2_category = mean_squared_error(combined_data_categorized_rf['Actual_AQI_SO2_Category'], 
                                      combined_data_categorized_rf['Predicted_AQI_SO2_Category_RF'], squared=False)

# 카테고리화된 데이터를 기반으로 결정 계수 계산
r2_no2_category = accuracy_score(combined_data_categorized_rf['Actual_AQI_NO2_Category'], 
                                combined_data_categorized_rf['Predicted_AQI_NO2_Category_RF'])

r2_so2_category = accuracy_score(combined_data_categorized_rf['Actual_AQI_SO2_Category'], 
                                combined_data_categorized_rf['Predicted_AQI_SO2_Category_RF'])

print(f"NO2 예측 모델의 카테고리별 결정 계수 (Random Forest) : {r2_no2_category:.4f}")
print(f"SO2 예측 모델의 카테고리별 결정 계수 (Random Forest) : {r2_so2_category:.4f}")
print(f"NO2 예측 모델의 카테고리별 RMSE (Random Forest) : {rmse_no2_category:.4f}")
print(f"SO2 예측 모델의 카테고리별 RMSE (Random Forest) : {rmse_so2_category:.4f}")


2024년 NO2 및 SO2 예측 적중률 (Random Forest)
2024년 동안 NO2의 예측 카테고리가 실제와 같은 비율 (Random Forest): 51.64%
2024년 동안 SO2의 예측 카테고리가 실제와 같은 비율 (Random Forest): 51.64%

2024-09-30 대기질 예측 데이터 (Random Forest)
예상 NO2 : 0 (Category: 1)
예상 SO2 : 0 (Category: 1)

NO2 예측 모델의 결정 계수 (Random Forest): 0.1423
SO2 예측 모델의 결정 계수 (Random Forest): 0.1167
NO2 예측 모델의 RMSE (Random Forest): 12.2394
SO2 예측 모델의 RMSE (Random Forest): 1.1439

NO2 예측 모델의 카테고리별 결정 계수 (Random Forest) : 0.5164
SO2 예측 모델의 카테고리별 결정 계수 (Random Forest) : 0.5164
NO2 예측 모델의 카테고리별 RMSE (Random Forest) : 2.0863
SO2 예측 모델의 카테고리별 RMSE (Random Forest) : 2.0863


,date,pm25,pm10,o3,no2,so2,co
0,2024-07-01,22,11,39,2,,
1,2024-07-02,25,25,63,2,8,
2,2024-07-03,50,20,59,2,12,
3,2024-07-04,53,14,46,1,,
4,2024-07-05,44,17,48,1,,
...,...,...,...,...,...,...,...
2096,2020-05-25,,22,40,7,2,
2097,2020-06-21,,18,33,2,1,
2098,2019-12-31,,46,23,14,6,
2099,2018-12-31,,51,21,16,6,


In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    numeric_df.fillna(0, inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# 예측 모델에 사용할 변수 및 대상 정의
pollutants = ['pm25', 'pm10', 'no2', 'so2', 'co', 'o3']
models = {}
X_train = {}
y_train = {}

# 각 오염물질에 대해 모델 준비
for pollutant in pollutants:
    X_train[pollutant] = corr_beijing_seoul[[f' {pollutant}_beijing' for pollutant in pollutants]]
    y_train[pollutant] = corr_beijing_seoul[f' {pollutant}_seoul']
    
    X_train[pollutant], y_train[pollutant] = prepare_data(X_train[pollutant], y_train[pollutant])
    models[pollutant] = LinearRegression()
    models[pollutant].fit(X_train[pollutant], y_train[pollutant])

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates
})

# 미래 데이터 생성
for pollutant in pollutants:
    future_beijing[f' {pollutant}_beijing'] = [random.randint(50, 150) if pollutant in ['pm25', 'pm10'] else random.randint(10, 50) for _ in range(len(future_dates))]

# 예측 수행
for pollutant in pollutants:
    future_X = future_beijing[[f' {pollutant}_beijing' for pollutant in pollutants]]
    future_beijing[f'Predicted_{pollutant.upper()}'] = models[pollutant].predict(future_X)

# AQI를 μg/m³로 변환하는 함수 정의 (한국 기준)
def aqi_conversion(pm_value, pm_type):
    if pm_type == 'PM2.5':
        if pm_value <= 12:
            return pm_value * (15.0 / 12)
        elif pm_value <= 35.4:
            return 15.0 + (pm_value - 12) * ((35.0 - 15.0) / (35.4 - 12))
        elif pm_value <= 75.4:
            return 35.0 + (pm_value - 35.4) * ((75.0 - 35.0) / (75.4 - 35.4))
        elif pm_value <= 115.4:
            return 75.0 + (pm_value - 75.4) * ((115.0 - 75.0) / (115.4 - 75.4))
        elif pm_value <= 150.4:
            return 115.0 + (pm_value - 115.4) * ((150.0 - 115.0) / (150.4 - 115.4))
        elif pm_value <= 500:
            return 150.0 + (pm_value - 150.4) * ((350.0 - 150.0) / (500 - 150.4))
        else:
            return 500.0
    elif pm_type == 'PM10':
        if pm_value <= 30:
            return pm_value * (30.0 / 30)
        elif pm_value <= 80:
            return 30.0 + (pm_value - 30) * ((80.0 - 30.0) / (80 - 30))
        elif pm_value <= 150:
            return 80.0 + (pm_value - 80) * ((150.0 - 80.0) / (150 - 80))
        elif pm_value <= 250:
            return 150.0 + (pm_value - 150) * ((250.0 - 150.0) / (250 - 150))
        elif pm_value <= 350:
            return 250.0 + (pm_value - 250) * ((350.0 - 250.0) / (350 - 250))
        elif pm_value <= 420:
            return 350.0 + (pm_value - 350) * ((420.0 - 350.0) / (420 - 350))
        else:
            return 420.0
    # You can define conversion functions for NO2, SO2, CO, O3 here if needed

# 예측 데이터를 AQI로 변환
for pollutant in pollutants:
    if pollutant in ['pm25', 'pm10']:  # Update conversion for other pollutants if needed
        future_beijing[f'Predicted_AQI_{pollutant.upper()}'] = future_beijing[f'Predicted_{pollutant.upper()}'].apply(lambda x: aqi_conversion(x, pollutant.upper()))

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')].reset_index(drop=True)
actual_seoul.columns = ['date'] + [f'Actual_{pollutant.upper()}' for pollutant in pollutants]

# 'Actual_PM25'와 'Actual_PM10' 열을 숫자로 변환
for pollutant in pollutants:
    actual_seoul[f'Actual_{pollutant.upper()}'] = pd.to_numeric(actual_seoul[f'Actual_{pollutant.upper()}'], errors='coerce')

# 데이터를 AQI로 변환
for pollutant in pollutants:
    if pollutant in ['pm25', 'pm10']:  # Update conversion for other pollutants if needed
        actual_seoul[f'Actual_AQI_{pollutant.upper()}'] = actual_seoul[f'Actual_{pollutant.upper()}'].apply(lambda x: aqi_conversion(x, pollutant.upper()))

# 오차 비율 계산
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')
combined_data
for pollutant in pollutants:
    combined_data[f'{pollutant.upper()}_Error_Rate'] = np.abs((combined_data[f'Actual_AQI_{pollutant.upper()}'] - combined_data[f'Predicted_AQI_{pollutant.upper()}']) / combined_data[f'Actual_AQI_{pollutant.upper()}']) * 100

# # 함수 정의: AQI 값을 입력받아 범주를 반환하는 함수
# def categorize_aqi(aqi_value):
#     if aqi_value <= 100:
#         return 1
#     elif aqi_value <= 200:
#         return 2
#     elif aqi_value <= 300:
#         return 3
#     else:
#         return 4

# # 예측된 AQI 값을 범주형으로 변환하는 함수 정의
# def categorize_predicted_aqi(df):
#     for pollutant in pollutants:
#         df[f'Predicted_AQI_{pollutant.upper()}_Category'] = df[f'Predicted_AQI_{pollutant.upper()}'].apply(categorize_aqi)
#     return df

# # 실제 AQI 값을 범주형으로 변환하는 함수 정의
# def categorize_actual_aqi(df):
#     for pollutant in pollutants:
#         df[f'Actual_AQI_{pollutant.upper()}_Category'] = df[f'Actual_AQI_{pollutant.upper()}'].apply(categorize_aqi)
#     return df

# # 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
# def compare_aqi_categories(row):
#     for pollutant in pollutants:
#         if row[f'Predicted_AQI_{pollutant.upper()}_Category'] != row[f'Actual_AQI_{pollutant.upper()}_Category']:
#             row[f'{pollutant.upper()}_Category_Difference'] = 1
#         else:
#             row[f'{pollutant.upper()}_Category_Difference'] = 0
#     return row

# # COMBINED_DATA에 적용
# combined_data_categorized = categorize_predicted_aqi(combined_data)
# combined_data_categorized = categorize_actual_aqi(combined_data_categorized)

# # combined_data_categorized에 함수 적용
# combined_data_categorized_diff = combined_data_categorized.apply(compare_aqi_categories, axis=1)

# # 결과 출력
# print("2024년 미세먼지 예측 적중률")

# # 2024년 한 해 동안의 데이터 필터링
# total_data = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
#                                            (combined_data_categorized_diff['date'] <= '2024-12-31')]

# # PM2.5 및 PM10의 카테고리가 같은 비율 계산
# total_days = len(total_data)
# same_category_count = {}
# percent_same_category = {}

# for pollutant in pollutants:
#     same_category_count[pollutant] = total_data[total_data[f'{pollutant.upper()}_Category_Difference'] == 0].shape[0]
#     percent_same_category[pollutant] = (same_category_count[pollutant] / total_days) * 100

# for pollutant in pollutants:
#     print(f"2024년 동안 {pollutant.upper()}의 예측 카테고리가 실제와 같은 비율: {percent_same_category[pollutant]:.2f}%")

# # 특정 날짜의 데이터 출력 함수
# def print_aqi_for_date(target_date):
#     target_date = pd.to_datetime(target_date)
#     row = combined_data[combined_data['date'] == target_date]
#     if row.empty:
#         print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
#     else:
#         print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터")
#         for pollutant in pollutants:
#             print(f"예상 {pollutant.upper()} : {round(row[f'Predicted_AQI_{pollutant.upper()}'].values[0])} (Category: {row[f'Predicted_AQI_{pollutant.upper()}_Category'].values[0]})")
#         print("\n")

# # 특정 날짜의 AQI 데이터를 출력
# target_date = '2024-12-20'
# print_aqi_for_date(target_date)

# # 예측값 계산
# y_pred = {}
# r2 = {}
# rmse = {}

# for pollutant in pollutants:
#     y_pred[pollutant] = models[pollutant].predict(X_train[pollutant])
#     r2[pollutant] = r2_score(y_train[pollutant], y_pred[pollutant])
#     rmse[pollutant] = mean_squared_error(y_train[pollutant], y_pred[pollutant], squared=False)
#     print(f"{pollutant.upper()} 예측 모델의 결정 계수 (R-squared): {r2[pollutant]:.4f}")
#     print(f"{pollutant.upper()} 예측 모델의 RMSE: {rmse[pollutant]:.4f}")

# # 예측된 AQI 카테고리 계산 함수
# def categorize_predicted_aqi_category(df):
#     for pollutant in pollutants:
#         df[f'Predicted_AQI_{pollutant.upper()}_Category'] = df[f'Predicted_AQI_{pollutant.upper()}'].apply(categorize_aqi)
#     return df

# # 카테고리화된 데이터 준비
# combined_data_categorized = categorize_predicted_aqi_category(combined_data)

# # 실제 AQI 카테고리 계산 함수
# def categorize_actual_aqi_category(df):
#     for pollutant in pollutants:
#         df[f'Actual_AQI_{pollutant.upper()}_Category'] = df[f'Actual_AQI_{pollutant.upper()}'].apply(categorize_aqi)
#     return df

# # 카테고리화된 실제 데이터 준비
# combined_data_categorized = categorize_actual_aqi_category(combined_data_categorized)

# # 카테고리화된 데이터를 기반으로 RMSE 계산
# rmse_category = {}
# for pollutant in pollutants:
#     rmse_category[pollutant] = mean_squared_error(combined_data_categorized[f'Actual_AQI_{pollutant.upper()}_Category'], 
#                                                   combined_data_categorized[f'Predicted_AQI_{pollutant.upper()}_Category'], squared=False)
#     print(f"{pollutant.upper()} 예측 모델의 카테고리별 RMSE: {rmse_category[pollutant]:.4f}")

# # 카테고리화된 데이터를 기반으로 결정 계수 계산
# # 실제 데이터가 있는 경우만 필터링
# filtered_data = combined_data_categorized[combined_data_categorized[f'Actual_AQI_PM25_Category'] != 4]

# # 카테고리별 결정 계수 계산
# accuracy_category = {}
# for pollutant in pollutants:
#     accuracy_category[pollutant] = accuracy_score(filtered_data[f'Actual_AQI_{pollutant.upper()}_Category'], 
#                                                   filtered_data[f'Predicted_AQI_{pollutant.upper()}_Category'])
#     print(f"{pollutant.upper()} 예측 모델의 카테고리별 결정 계수 (필터링 후): {accuracy_category[pollutant]:.4f}")



In [9]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from datetime import datetime

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    numeric_df.fillna(0, inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# 예측 모델에 사용할 변수 및 대상 정의
pollutants = ['pm25', 'pm10', 'no2', 'so2', 'co', 'o3']
models = {}
X_train = {}
y_train = {}

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# 각 오염물질에 대해 모델 준비
for pollutant in pollutants:
    X_train[pollutant] = corr_beijing_seoul[[f' {pollutant}_beijing' for pollutant in pollutants]]
    y_train[pollutant] = corr_beijing_seoul[f' {pollutant}_seoul']
    
    X_train[pollutant], y_train[pollutant] = prepare_data(X_train[pollutant], y_train[pollutant])
    models[pollutant] = LinearRegression()
    models[pollutant].fit(X_train[pollutant], y_train[pollutant])

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates
})

# 미래 데이터 생성
for pollutant in pollutants:
    future_beijing[f' {pollutant}_beijing'] = [random.randint(50, 150) if pollutant in ['pm25', 'pm10'] else random.randint(10, 50) for _ in range(len(future_dates))]

# 예측 수행
for pollutant in pollutants:
    future_X = future_beijing[[f' {pollutant}_beijing' for pollutant in pollutants]]
    future_beijing[f'Predicted_{pollutant.upper()}'] = models[pollutant].predict(future_X)

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')].reset_index(drop=True)
actual_seoul.columns = ['date'] + [f'Actual_{pollutant.upper()}' for pollutant in pollutants]

# 'Actual_PM25'와 'Actual_PM10' 열을 숫자로 변환
for pollutant in pollutants:
    actual_seoul[f'Actual_{pollutant.upper()}'] = pd.to_numeric(actual_seoul[f'Actual_{pollutant.upper()}'], errors='coerce')

# 오차 비율 계산
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')

for pollutant in pollutants:
    combined_data[f'{pollutant.upper()}_Error_Rate'] = np.abs((combined_data[f'Actual_{pollutant.upper()}'] - combined_data[f'Predicted_{pollutant.upper()}']) / combined_data[f'Actual_{pollutant.upper()}']) * 100

# 함수 정의: 오염 물질 값을 입력받아 범주를 반환하는 함수
def categorize_pollutant(value, thresholds):
    for i, threshold in enumerate(thresholds):
        if value <= threshold:
            return i + 1
    return len(thresholds) + 1

# 예측된 오염 물질 값을 범주형으로 변환하는 함수 정의
def categorize_predicted_values(df):
    thresholds = {
        'pm25': [100, 200, 300, 400],
        'pm10': [100, 200, 300, 400],
        'no2': [100, 200, 300, 400],
        'so2': [100, 200, 300, 400],
        'co': [100, 200, 300, 400],
        'o3': [100, 200, 300, 400]
    }
    for pollutant in pollutants:
        df[f'Predicted_{pollutant.upper()}_Category'] = df[f'Predicted_{pollutant.upper()}'].apply(lambda x: categorize_pollutant(x, thresholds[pollutant]))
    return df

# 실제 오염 물질 값을 범주형으로 변환하는 함수 정의
def categorize_actual_values(df):
    thresholds = {
        'pm25': [100, 200, 300, 400],
        'pm10': [100, 200, 300, 400],
        'no2': [100, 200, 300, 400],
        'so2': [100, 200, 300, 400],
        'co': [100, 200, 300, 400],
        'o3': [100, 200, 300, 400]
    }
    for pollutant in pollutants:
        df[f'Actual_{pollutant.upper()}_Category'] = df[f'Actual_{pollutant.upper()}'].apply(lambda x: categorize_pollutant(x, thresholds[pollutant]))
    return df

# 예측된 카테고리와 실제 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_categories(row):
    for pollutant in pollutants:
        if row[f'Predicted_{pollutant.upper()}_Category'] != row[f'Actual_{pollutant.upper()}_Category']:
            row[f'{pollutant.upper()}_Category_Difference'] = 1
        else:
            row[f'{pollutant.upper()}_Category_Difference'] = 0
    return row

# COMBINED_DATA에 적용
combined_data_categorized = categorize_predicted_values(combined_data)
combined_data_categorized = categorize_actual_values(combined_data_categorized)

# combined_data_categorized에 함수 적용
combined_data_categorized_diff = combined_data_categorized.apply(compare_categories, axis=1)

# 결과 출력
print("2024년 미세먼지 예측 적중률")

# 2024년 한 해 동안의 데이터 필터링
total_data = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-12-31')]

# PM2.5 및 PM10의 카테고리가 같은 비율 계산
total_days = len(total_data)
same_category_count = {}
percent_same_category = {}

# 현재 날짜 가져오기
current_date = datetime.now()

# 현재 날짜를 기준으로 데이터 필터링
total_data_current = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                                   (combined_data_categorized_diff['date'] <= current_date)].reset_index(drop=True)

# 필터링된 기간 동안의 데이터 수
total_days_current = len(total_data_current)

# 카테고리가 같은 비율 계산
percent_same_category_current = {}

for pollutant in pollutants:
    same_category_count[pollutant] = total_data_current[total_data_current[f'{pollutant.upper()}_Category_Difference'] == 0].shape[0]
    percent_same_category_current[pollutant] = (same_category_count[pollutant] / total_days_current) * 100

# 결과 출력
for pollutant in pollutants:
    print(f"현재까지 {pollutant.upper()}의 예측 카테고리가 실제와 같은 비율: {percent_same_category_current[pollutant]:.2f}%")



2024년 미세먼지 예측 적중률
현재까지 PM25의 예측 카테고리가 실제와 같은 비율: 37.50%
현재까지 PM10의 예측 카테고리가 실제와 같은 비율: 96.88%
현재까지 NO2의 예측 카테고리가 실제와 같은 비율: 95.83%
현재까지 SO2의 예측 카테고리가 실제와 같은 비율: 97.92%
현재까지 CO의 예측 카테고리가 실제와 같은 비율: 97.92%
현재까지 O3의 예측 카테고리가 실제와 같은 비율: 97.92%
2024-12-20 대기질 예측 데이터
예상 PM25 : 126 (Category: 2)
예상 PM10 : 62 (Category: 1)
예상 NO2 : 58 (Category: 1)
예상 SO2 : 10 (Category: 1)
예상 CO : 11 (Category: 1)
예상 O3 : -5 (Category: 1)


PM25 예측 모델의 결정 계수 (R-squared): 0.0573
PM25 예측 모델의 RMSE: 33.9278
PM10 예측 모델의 결정 계수 (R-squared): 0.0679
PM10 예측 모델의 RMSE: 21.3740
NO2 예측 모델의 결정 계수 (R-squared): 0.0638
NO2 예측 모델의 RMSE: 12.7869
SO2 예측 모델의 결정 계수 (R-squared): 0.0550
SO2 예측 모델의 RMSE: 1.1832
CO 예측 모델의 결정 계수 (R-squared): 0.0402
CO 예측 모델의 RMSE: 2.1100
O3 예측 모델의 결정 계수 (R-squared): 0.0363
O3 예측 모델의 RMSE: 16.5560
PM25 예측 모델의 카테고리별 RMSE: 2.2694
PM10 예측 모델의 카테고리별 RMSE: 2.7905
NO2 예측 모델의 카테고리별 RMSE: 2.7915
SO2 예측 모델의 카테고리별 RMSE: 2.7895
CO 예측 모델의 카테고리별 RMSE: 2.7895
O3 예측 모델의 카테고리별 RMSE: 2.7895
PM25 예측 모델의 카테고리별 결정 계수 (필터링 

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    numeric_df.fillna(0, inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# 예측 모델에 사용할 변수 및 대상 정의
pollutants = ['pm25', 'pm10', 'co', 'o3']
models = {}
X_train = {}
y_train = {}

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# 각 오염물질에 대해 모델 준비
for pollutant in pollutants:
    X_train[pollutant] = corr_beijing_seoul[[f' {pollutant}_beijing' for pollutant in pollutants]]
    y_train[pollutant] = corr_beijing_seoul[f' {pollutant}_seoul']
    
    X_train[pollutant], y_train[pollutant] = prepare_data(X_train[pollutant], y_train[pollutant])
    models[pollutant] = LinearRegression()
    models[pollutant].fit(X_train[pollutant], y_train[pollutant])

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates
})

# 미래 데이터 생성
for pollutant in pollutants:
    future_beijing[f' {pollutant}_beijing'] = [random.randint(50, 150) if pollutant in ['pm25', 'pm10'] else random.randint(10, 50) for _ in range(len(future_dates))]

# 예측 수행
for pollutant in pollutants:
    future_X = future_beijing[[f' {pollutant}_beijing' for pollutant in pollutants]]
    future_beijing[f'Predicted_{pollutant.upper()}'] = models[pollutant].predict(future_X)

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')].reset_index(drop=True)
actual_seoul.columns = ['date'] + [f'Actual_{pollutant.upper()}' for pollutant in pollutants]

# 'Actual_PM25'와 'Actual_PM10' 열을 숫자로 변환
for pollutant in pollutants:
    actual_seoul[f'Actual_{pollutant.upper()}'] = pd.to_numeric(actual_seoul[f'Actual_{pollutant.upper()}'], errors='coerce')

# 오차 비율 계산
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')

for pollutant in pollutants:
    combined_data[f'{pollutant.upper()}_Error_Rate'] = np.abs((combined_data[f'Actual_{pollutant.upper()}'] - combined_data[f'Predicted_{pollutant.upper()}']) / combined_data[f'Actual_{pollutant.upper()}']) * 100

# 함수 정의: 오염 물질 값을 입력받아 범주를 반환하는 함수
def categorize_pollutant(value, thresholds):
    for i, threshold in enumerate(thresholds):
        if value <= threshold:
            return i + 1
    return len(thresholds) + 1

# 예측된 오염 물질 값을 범주형으로 변환하는 함수 정의
def categorize_predicted_values(df):
    thresholds = {
        'pm25': [12, 35.4, 55.4, 150.4],
        'pm10': [54, 154, 254, 354],
        'co': [4.4, 9.4, 12.4, 15.4],
        'o3': [54, 70, 85, 105]
    }
    for pollutant in pollutants:
        df[f'Predicted_{pollutant.upper()}_Category'] = df[f'Predicted_{pollutant.upper()}'].apply(lambda x: categorize_pollutant(x, thresholds[pollutant]))
    return df

# 실제 오염 물질 값을 범주형으로 변환하는 함수 정의
def categorize_actual_values(df):
    thresholds = {
        'pm25': [12, 35.4, 55.4, 150.4],
        'pm10': [54, 154, 254, 354],
        'co': [4.4, 9.4, 12.4, 15.4],
        'o3': [54, 70, 85, 105]
    }
    for pollutant in pollutants:
        df[f'Actual_{pollutant.upper()}_Category'] = df[f'Actual_{pollutant.upper()}'].apply(lambda x: categorize_pollutant(x, thresholds[pollutant]))
    return df

# 예측된 카테고리와 실제 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_categories(row):
    for pollutant in pollutants:
        if row[f'Predicted_{pollutant.upper()}_Category'] != row[f'Actual_{pollutant.upper()}_Category']:
            row[f'{pollutant.upper()}_Category_Difference'] = 1
        else:
            row[f'{pollutant.upper()}_Category_Difference'] = 0
    return row

# COMBINED_DATA에 적용
combined_data_categorized = categorize_predicted_values(combined_data)
combined_data_categorized = categorize_actual_values(combined_data_categorized)

# combined_data_categorized에 함수 적용
combined_data_categorized_diff = combined_data_categorized.apply(compare_categories, axis=1)

# 결과 출력
print("2024년 미세먼지 예측 적중률")

# 2024년 한 해 동안의 데이터 필터링
total_data = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-12-31')]

# PM2.5 및 PM10의 카테고리가 같은 비율 계산
total_days = len(total_data)
same_category_count = {}
percent_same_category = {}

for pollutant in pollutants:
    same_category_count[pollutant] = total_data[total_data[f'{pollutant.upper()}_Category_Difference'] == 0].shape[0]
    percent_same_category[pollutant] = (same_category_count[pollutant] / total_days) * 100

for pollutant in pollutants:
    print(f"2024년 동안 {pollutant.upper()}의 예측 카테고리가 실제와 같은 비율: {percent_same_category[pollutant]:.2f}%")

# 특정 날짜의 데이터 출력 함수
def print_pollutant_for_date(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data[combined_data['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터")
        for pollutant in pollutants:
            print(f"예상 {pollutant.upper()} : {round(row[f'Predicted_{pollutant.upper()}'].values[0])} (Category: {row[f'Predicted_{pollutant.upper()}_Category'].values[0]})")
        actual_values = {}
        actual_categories = {}
        for pollutant in pollutants:
            actual_values[pollutant] = round(row[f'Actual_{pollutant.upper()}'].values[0]) if not np.isnan(row[f'Actual_{pollutant.upper()}'].values[0]) else 'N/A'
            actual_categories[pollutant] = row[f'Actual_{pollutant.upper()}_Category'].values[0] if not np.isnan(row[f'Actual_{pollutant.upper()}'].values[0]) else 'N/A'
            print(f"실제 {pollutant.upper()} : {actual_values[pollutant]} (Category: {actual_categories[pollutant]})")

# 특정 날짜의 데이터를 출력
target_date = '2024-12-20'
print_pollutant_for_date(target_date)

# 예측값 계산
y_pred = {}
r2 = {}
rmse = {}

for pollutant in pollutants:
    y_pred[pollutant] = models[pollutant].predict(X_train[pollutant])
    r2[pollutant] = r2_score(y_train[pollutant], y_pred[pollutant])
    rmse[pollutant] = mean_squared_error(y_train[pollutant], y_pred[pollutant], squared=False)
    print(f"{pollutant.upper()} 예측 모델의 결정 계수 (R-squared): {r2[pollutant]:.4f}")
    print(f"{pollutant.upper()} 예측 모델의 RMSE: {rmse[pollutant]:.4f}")

# 예측된 카테고리 계산 함수
def categorize_predicted_values_category(df):
    thresholds = {
        'pm25': [12, 35.4, 55.4, 150.4],
        'pm10': [54, 154, 254, 354],
        'co': [4.4, 9.4, 12.4, 15.4],
        'o3': [54, 70, 85, 105]
    }
    for pollutant in pollutants:
        df[f'Predicted_{pollutant.upper()}_Category'] = df[f'Predicted_{pollutant.upper()}'].apply(lambda x: categorize_pollutant(x, thresholds[pollutant]))
    return df

# 카테고리화된 데이터 준비
combined_data_categorized = categorize_predicted_values_category(combined_data)

# 실제 카테고리 계산 함수
def categorize_actual_values_category(df):
    thresholds = {
        'pm25': [12, 35.4, 55.4, 150.4],
        'pm10': [54, 154, 254, 354],
        'co': [4.4, 9.4, 12.4, 15.4],
        'o3': [54, 70, 85, 105]
    }
    for pollutant in pollutants:
        df[f'Actual_{pollutant.upper()}_Category'] = df[f'Actual_{pollutant.upper()}'].apply(lambda x: categorize_pollutant(x, thresholds[pollutant]))
    return df

# 카테고리화된 실제 데이터 준비
combined_data_categorized = categorize_actual_values_category(combined_data_categorized)

# 카테고리화된 데이터를 기반으로 RMSE 계산
rmse_category = {}
for pollutant in pollutants:
    rmse_category[pollutant] = mean_squared_error(combined_data_categorized[f'Actual_{pollutant.upper()}_Category'], 
                                                  combined_data_categorized[f'Predicted_{pollutant.upper()}_Category'], squared=False)
    print(f"{pollutant.upper()} 예측 모델의 카테고리별 RMSE: {rmse_category[pollutant]:.4f}")

# 카테고리화된 데이터를 기반으로 결정 계수 계산
# 실제 데이터가 있는 경우만 필터링
filtered_data = combined_data_categorized[combined_data_categorized[f'Actual_PM25_Category'] != 4]

# 카테고리별 결정 계수 계산
accuracy_category = {}
for pollutant in pollutants:
    accuracy_category[pollutant] = accuracy_score(filtered_data[f'Actual_{pollutant.upper()}_Category'], 
                                                  filtered_data[f'Predicted_{pollutant.upper()}_Category'])
    print(f"{pollutant.upper()} 예측 모델의 카테고리별 결정 계수 (필터링 후): {accuracy_category[pollutant]:.4f}")


In [66]:
beijing_numeric

,pm25,pm10,o3,no2,so2,co
189,28.000000,13.0,39.0,8.0,3.0,0.0
190,31.000000,19.0,39.0,7.0,3.0,0.0
191,53.000000,24.0,46.0,6.0,3.0,0.0
192,61.000000,30.0,28.0,5.0,4.0,0.0
193,21.000000,23.0,32.0,9.0,4.0,0.0
...,...,...,...,...,...,...
2096,67.390608,22.0,40.0,7.0,2.0,0.0
2097,67.390608,18.0,33.0,2.0,1.0,0.0
2098,67.390608,46.0,23.0,14.0,6.0,0.0
2099,67.390608,51.0,21.0,16.0,6.0,0.0


In [147]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import accuracy_score
# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

def preprocess(df):
    # date 컬럼을 제외한 나머지 컬럼들을 숫자로 변환
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    
    # 공백을 NaN으로 대체
    numeric_df.replace(' ', np.nan, inplace=True)
    
    # NaN 값을 각 컬럼의 평균 값으로 대체
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    
    # 남아있는 NaN 값들을 0으로 대체
    numeric_df.fillna(0, inplace=True)
    
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))
corr_beijing_seoul['date'] = seoul_past['date']

# PM2.5 예측 모델
X_pm25 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm25 = corr_beijing_seoul[' pm25_seoul']

# PM10 예측 모델
X_pm10 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm10 = corr_beijing_seoul[' pm10_seoul']

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# PM2.5, PM10 데이터 준비
X_pm25, y_pm25 = prepare_data(X_pm25, y_pm25)
X_pm10, y_pm10 = prepare_data(X_pm10, y_pm10)

# 모델 생성 및 학습
model_pm25 = LinearRegression()
model_pm25.fit(X_pm25, y_pm25)

model_pm10 = LinearRegression()
model_pm10.fit(X_pm10, y_pm10)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    ' pm25_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    ' pm10_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# PM2.5, PM10 예측
future_X_pm25 = future_beijing[[' pm25_beijing', ' pm10_beijing']]
future_X_pm10 = future_beijing[[' pm25_beijing', ' pm10_beijing']]

future_pred_pm25 = model_pm25.predict(future_X_pm25)
future_pred_pm10 = model_pm10.predict(future_X_pm10)

future_beijing['Predicted_PM25'] = future_pred_pm25
future_beijing['Predicted_PM10'] = future_pred_pm10

# AQI를 μg/m³로 변환하는 함수 정의 (한국 기준)
def aqi_conversion(pm_value, pm_type):
    if pm_type == 'PM2.5':
        if pm_value <= 12:
            return pm_value * (15.0 / 12)
        elif pm_value <= 35.4:
            return 15.0 + (pm_value - 12) * ((35.0 - 15.0) / (35.4 - 12))
        elif pm_value <= 75.4:
            return 35.0 + (pm_value - 35.4) * ((75.0 - 35.0) / (75.4 - 35.4))
        elif pm_value <= 115.4:
            return 75.0 + (pm_value - 75.4) * ((115.0 - 75.0) / (115.4 - 75.4))
        elif pm_value <= 150.4:
            return 115.0 + (pm_value - 115.4) * ((150.0 - 115.0) / (150.4 - 115.4))
        elif pm_value <= 500:
            return 150.0 + (pm_value - 150.4) * ((350.0 - 150.0) / (500 - 150.4))
        else:
            return 500.0
    elif pm_type == 'PM10':
        if pm_value <= 30:
            return pm_value * (30.0 / 30)
        elif pm_value <= 80:
            return 30.0 + (pm_value - 30) * ((80.0 - 30.0) / (80 - 30))
        elif pm_value <= 150:
            return 80.0 + (pm_value - 80) * ((150.0 - 80.0) / (150 - 80))
        elif pm_value <= 250:
            return 150.0 + (pm_value - 150) * ((250.0 - 150.0) / (250 - 150))
        elif pm_value <= 350:
            return 250.0 + (pm_value - 250) * ((350.0 - 250.0) / (350 - 250))
        elif pm_value <= 420:
            return 350.0 + (pm_value - 350) * ((420.0 - 350.0) / (420 - 350))
        else:
            return 420.0

# 예측 데이터를 AQI로 변환
future_beijing['Predicted_AQI_PM25'] = future_beijing['Predicted_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
future_beijing['Predicted_AQI_PM10'] = future_beijing['Predicted_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' pm25', ' pm10']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_PM25', 'Actual_PM10']

# 'Actual_PM25'와 'Actual_PM10' 열을 숫자로 변환
actual_seoul['Actual_PM25'] = pd.to_numeric(actual_seoul['Actual_PM25'], errors='coerce')
actual_seoul['Actual_PM10'] = pd.to_numeric(actual_seoul['Actual_PM10'], errors='coerce')

# 데이터를 AQI로 변환
actual_seoul['Actual_AQI_PM25'] = actual_seoul['Actual_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
actual_seoul['Actual_AQI_PM10'] = actual_seoul['Actual_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 오차 비율 계산
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')
combined_data['PM25_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM25'] - combined_data['Predicted_AQI_PM25']) / combined_data['Actual_AQI_PM25']) * 100
combined_data['PM10_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM10'] - combined_data['Predicted_AQI_PM10']) / combined_data['Actual_AQI_PM10']) * 100

# 함수 정의: AQI 값을 입력받아 범주를 반환하는 함수
def categorize_aqi(aqi_value):
    if aqi_value <= 100:
        return 1
    elif aqi_value <= 200:
        return 2
    elif aqi_value <= 300:
        return 3
    else:
        return 4

# 예측된 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_predicted_aqi(df):
    df['Predicted_AQI_PM25_Category'] = df['Predicted_AQI_PM25'].apply(categorize_aqi)
    df['Predicted_AQI_PM10_Category'] = df['Predicted_AQI_PM10'].apply(categorize_aqi)
    return df

# 실제 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_actual_aqi(df):
    df['Actual_AQI_PM25_Category'] = df['Actual_AQI_PM25'].apply(categorize_aqi)
    df['Actual_AQI_PM10_Category'] = df['Actual_AQI_PM10'].apply(categorize_aqi)
    return df

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories(row):
    if row['Predicted_AQI_PM25_Category'] != row['Actual_AQI_PM25_Category']:
        row['PM25_Category_Difference'] = 1
    else:
        row['PM25_Category_Difference'] = 0
    
    if row['Predicted_AQI_PM10_Category'] != row['Actual_AQI_PM10_Category']:
        row['PM10_Category_Difference'] = 1
    else:
        row['PM10_Category_Difference'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_categorized = categorize_predicted_aqi(combined_data)
combined_data_categorized = categorize_actual_aqi(combined_data_categorized)

# combined_data_categorized에 함수 적용
combined_data_categorized_diff = combined_data_categorized.apply(compare_aqi_categories, axis=1)

# 결과 출력
print("2024년 미세먼지 예측 적중률")
# combined_data_categorized_diff[['date', 'Predicted_AQI_PM25_Category', 'Actual_AQI_PM25_Category', 
#                                       'PM25_Category_Difference', 'Predicted_AQI_PM10_Category', 
#                                       'Actual_AQI_PM10_Category', 'PM10_Category_Difference']]

# 2024년 한 해 동안의 데이터 필터링
total_data = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-12-31')]

year_data = total_data[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-06-30')]

# PM2.5 및 PM10의 카테고리가 같은 비율 계산
total_days = len(year_data)
same_pm25_category_count = year_data[year_data['PM25_Category_Difference'] == 0].shape[0]
same_pm10_category_count = year_data[year_data['PM10_Category_Difference'] == 0].shape[0]

percent_same_pm25_category = (same_pm25_category_count / total_days) * 100
percent_same_pm10_category = (same_pm10_category_count / total_days) * 100

# 결과 출력
print(f"2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: {percent_same_pm25_category:.2f}%")
print(f"2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: {percent_same_pm10_category:.2f}%\n")

# 특정 날짜의 데이터 출력 함수
def print_aqi_for_date(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data[combined_data['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터")
        print(f"예상 PM25 : {round(row['Predicted_AQI_PM25'].values[0])} (Category: {row['Predicted_AQI_PM25_Category'].values[0]})")
        print(f"예상 PM10 : {round(row['Predicted_AQI_PM10'].values[0])} (Category: {row['Predicted_AQI_PM10_Category'].values[0]})\n")

# 특정 날짜의 AQI 데이터를 출력
target_date = '2024-12-20'
print_aqi_for_date(target_date)

# 예측값 계산
y_pred_pm25 = model_pm25.predict(X_pm25)
y_pred_pm10 = model_pm10.predict(X_pm10)

# R-squared 계산
r2_pm25 = r2_score(y_pm25, y_pred_pm25)
r2_pm10 = r2_score(y_pm10, y_pred_pm10)

print(f"PM25 예측 모델의 결정 계수 (R-squared): {r2_pm25:.4f}")
print(f"PM10 예측 모델의 결정 계수 (R-squared): {r2_pm10:.4f}")

# RMSE 계산
rmse_pm25 = mean_squared_error(y_pm25, y_pred_pm25, squared=False)
rmse_pm10 = mean_squared_error(y_pm10, y_pred_pm10, squared=False)

print(f"PM2.5 예측 모델의 RMSE: {rmse_pm25:.4f}")
print(f"PM10 예측 모델의 RMSE: {rmse_pm10:.4f}\n")


# 예측된 AQI 카테고리 계산 함수
def categorize_predicted_aqi_category(df):
    df['Predicted_AQI_PM25_Category'] = df['Predicted_AQI_PM25'].apply(categorize_aqi)
    df['Predicted_AQI_PM10_Category'] = df['Predicted_AQI_PM10'].apply(categorize_aqi)
    return df

# 카테고리화된 데이터 준비
combined_data_categorized = categorize_predicted_aqi_category(combined_data)

# 실제 AQI 카테고리 계산 함수
def categorize_actual_aqi_category(df):
    df['Actual_AQI_PM25_Category'] = df['Actual_AQI_PM25'].apply(categorize_aqi)
    df['Actual_AQI_PM10_Category'] = df['Actual_AQI_PM10'].apply(categorize_aqi)
    return df

# 카테고리화된 실제 데이터 준비
combined_data_categorized = categorize_actual_aqi_category(combined_data_categorized)

# 카테고리화된 데이터를 기반으로 RMSE 계산
rmse_pm25_category = mean_squared_error(combined_data_categorized['Actual_AQI_PM25_Category'], 
                                        combined_data_categorized['Predicted_AQI_PM25_Category'], squared=False)

rmse_pm10_category = mean_squared_error(combined_data_categorized['Actual_AQI_PM10_Category'], 
                                        combined_data_categorized['Predicted_AQI_PM10_Category'], squared=False)

print(f"PM25 예측 모델의 카테고리별 RMSE: {rmse_pm25_category:.4f}")
print(f"PM10 예측 모델의 카테고리별 RMSE: {rmse_pm10_category:.4f}")

# 카테고리화된 데이터를 기반으로 결정 계수 계산

# 실제 데이터가 있는 경우만 필터링
filtered_data = combined_data_categorized[combined_data_categorized['Actual_AQI_PM25_Category'] != 4]

# PM2.5 예측 모델의 카테고리별 결정 계수 계산
r2_pm25_category_filtered = accuracy_score(filtered_data['Actual_AQI_PM25_Category'], 
                                          filtered_data['Predicted_AQI_PM25_Category'])

# PM10 예측 모델의 카테고리별 결정 계수 계산
r2_pm10_category_filtered = accuracy_score(filtered_data['Actual_AQI_PM10_Category'], 
                                          filtered_data['Predicted_AQI_PM10_Category'])

print(f"PM25 예측 모델의 카테고리별 결정 계수 (필터링 후): {r2_pm25_category_filtered:.4f}")
print(f"PM10 예측 모델의 카테고리별 결정 계수 (필터링 후): {r2_pm10_category_filtered:.4f}")

# CO 예측 모델
X_co = corr_beijing_seoul[[' co_beijing', ' o3_beijing']]
y_co = corr_beijing_seoul[' co_seoul']

# O3 예측 모델
X_o3 = corr_beijing_seoul[[' co_beijing', ' o3_beijing']]
y_o3 = corr_beijing_seoul[' o3_seoul']

# CO, O3 데이터 준비
X_co, y_co = prepare_data(X_co, y_co)
X_o3, y_o3 = prepare_data(X_o3, y_o3)

# 모델 생성 및 학습
model_co = LinearRegression()
model_co.fit(X_co, y_co)

model_o3 = LinearRegression()
model_o3.fit(X_o3, y_o3)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates_co = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing_co = pd.DataFrame({
    'date': future_dates,
    ' co_beijing': [random.randint(50, 150) for _ in range(len(future_dates_co))],
    ' o3_beijing': [random.randint(30, 100) for _ in range(len(future_dates_co))]
})

# CO, O3 예측
future_X_co = future_beijing_co[[' co_beijing', ' o3_beijing']]
future_X_o3 = future_beijing_co[[' co_beijing', ' o3_beijing']]

future_pred_co = model_co.predict(future_X_co)
future_pred_o3 = model_o3.predict(future_X_o3)

future_beijing_co['Predicted_CO'] = future_pred_co
future_beijing_co['Predicted_O3'] = future_pred_o3

# 실제 데이터를 가져옴 (서울)
actual_seoul_co = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' co', ' o3']].reset_index(drop=True)
actual_seoul_co.columns = ['date', 'Actual_CO', 'Actual_O3']

# 'Actual_CO'와 'Actual_O3' 열을 숫자로 변환
actual_seoul_co['Actual_CO'] = pd.to_numeric(actual_seoul_co['Actual_CO'], errors='coerce')
actual_seoul_co['Actual_O3'] = pd.to_numeric(actual_seoul_co['Actual_O3'], errors='coerce')

# 데이터 병합
combined_data_co = pd.merge(future_beijing_co, actual_seoul_co, on='date', how='left')

combined_data_co['CO_Predict_Category'] = combined_data_co['Predicted_CO'].apply(categorize_aqi)
combined_data_co['O3_Predict_Category'] = combined_data_co['Predicted_O3'].apply(categorize_aqi)
combined_data_co['CO_Actual_Category'] = combined_data_co['Actual_CO'].apply(categorize_aqi)
combined_data_co['O3_Actual_Category'] = combined_data_co['Actual_O3'].apply(categorize_aqi)

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories_co(row):
    if row['CO_Predict_Category'] != row['CO_Actual_Category']:
        row['CO_Category_Difference'] = 1
    else:
        row['CO_Category_Difference'] = 0
    
    if row['O3_Predict_Category'] != row['O3_Actual_Category']:
        row['O3_Category_Difference'] = 1
    else:
        row['O3_Category_Difference'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_diff_co = combined_data_co.apply(compare_aqi_categories_co, axis=1)
combined_data_co = pd.merge(combined_data_diff_co, combined_data_categorized_diff)

# 2024년 한 해 동안의 데이터 필터링
total_data_co = combined_data_co[(combined_data_co['date'] >= '2024-01-01') & 
                                           (combined_data_co['date'] <= '2024-12-31')]

year_data_co = total_data_co[(combined_data_co['date'] >= '2024-01-01') & 
                                           (combined_data_co['date'] <= '2024-06-30')]

# # PM2.5 및 PM10의 카테고리가 같은 비율 계산
same_co_category_count = year_data_co[year_data_co['CO_Category_Difference'] == 0].shape[0]
same_o3_category_count = year_data_co[year_data_co['O3_Category_Difference'] == 0].shape[0]

percent_same_co_category = (same_co_category_count / total_days) * 100
percent_same_o3_category = (same_o3_category_count / total_days) * 100

# # 결과 출력
print(f"\n\n2024년 동안 CO의 예측 카테고리가 실제와 같은 비율: {percent_same_co_category:.2f}%")
print(f"2024년 동안 O3의 예측 카테고리가 실제와 같은 비율: {percent_same_o3_category:.2f}%\n")

# 특정 날짜의 AQI 데이터를 출력 함수
def print_aqi_for_date_o3(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data_co[combined_data_co['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터")
        print(f"예상 CO : {round(row['Predicted_CO'].values[0])} (Category: {row['CO_Predict_Category'].values[0]})")
        print(f"예상 O3 : {round(row['Predicted_O3'].values[0])} (Category: {row['O3_Predict_Category'].values[0]})")
        actual_co = round(row['Actual_CO'].values[0]) if not np.isnan(row['Actual_CO'].values[0]) else 'N/A'
        actual_o3 = round(row['Actual_O3'].values[0]) if not np.isnan(row['Actual_O3'].values[0]) else 'N/A'
        actual_co_category = row['CO_Actual_Category'].values[0] if not np.isnan(row['Actual_CO'].values[0]) else 'N/A'
        actual_o3_category = row['O3_Actual_Category'].values[0] if not np.isnan(row['Actual_O3'].values[0]) else 'N/A'
        print(f"실제 CO : {actual_co} (Category: {actual_co_category})")
        print(f"실제 O3 : {actual_o3} (Category: {actual_o3_category})")

# 특정 날짜의 AQI 데이터를 출력
print_aqi_for_date_o3(target_date)

# NO2 예측 모델
X_no2 = corr_beijing_seoul[[' no2_beijing', ' so2_beijing']]
y_no2 = corr_beijing_seoul[' no2_seoul']

# SO2 예측 모델
X_so2 = corr_beijing_seoul[[' no2_beijing', ' so2_beijing']]
y_so2 = corr_beijing_seoul[' so2_seoul']

# NO2, SO2 데이터 준비
X_no2, y_no2 = prepare_data(X_no2, y_no2)
X_so2, y_so2 = prepare_data(X_so2, y_so2)

# 모델 생성 및 학습
model_no2 = LinearRegression()
model_no2.fit(X_no2, y_no2)

model_so2 = LinearRegression()
model_so2.fit(X_so2, y_so2)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates_ns = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing_ns = pd.DataFrame({
    'date': future_dates_ns,
    ' no2_beijing': [random.randint(0, 35) for _ in range(len(future_dates_ns))],
    ' so2_beijing': [random.randint(0, 15) for _ in range(len(future_dates_ns))]
})

# NO2, SO2 예측
future_X_no2 = future_beijing_ns[[' no2_beijing', ' so2_beijing']]
future_X_so2 = future_beijing_ns[[' no2_beijing', ' so2_beijing']]

future_pred_no2 = model_no2.predict(future_X_no2)
future_pred_so2 = model_so2.predict(future_X_so2)

future_beijing_ns['Predicted_NO2'] = future_pred_no2
future_beijing_ns['Predicted_SO2'] = future_pred_so2

# 실제 데이터를 가져옴 (서울)
actual_seoul_ns = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' no2', ' so2']].reset_index(drop=True)
actual_seoul_ns.columns = ['date', 'Actual_NO2', 'Actual_SO2']

# 'Actual_NO2'와 'Actual_SO2' 열을 숫자로 변환
actual_seoul_ns['Actual_NO2'] = pd.to_numeric(actual_seoul_ns['Actual_NO2'], errors='coerce')
actual_seoul_ns['Actual_SO2'] = pd.to_numeric(actual_seoul_ns['Actual_SO2'], errors='coerce')

# 데이터 병합
combined_data_ns = pd.merge(future_beijing_ns, actual_seoul_ns, on='date', how='left')

combined_data_ns['NO2_Predict_Category'] = combined_data_ns['Predicted_NO2'].apply(categorize_aqi)
combined_data_ns['SO2_Predict_Category'] = combined_data_ns['Predicted_SO2'].apply(categorize_aqi)
combined_data_ns['NO2_Actual_Category'] = combined_data_ns['Actual_NO2'].apply(categorize_aqi)
combined_data_ns['SO2_Actual_Category'] = combined_data_ns['Actual_SO2'].apply(categorize_aqi)

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories_ns(row):
    if row['NO2_Predict_Category'] != row['NO2_Actual_Category']:
        row['NO2_Category_Difference'] = 1
    else:
        row['NO2_Category_Difference'] = 0
    
    if row['SO2_Predict_Category'] != row['SO2_Actual_Category']:
        row['SO2_Category_Difference'] = 1
    else:
        row['SO2_Category_Difference'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_diff_ns = combined_data_ns.apply(compare_aqi_categories_ns, axis=1)
combined_data_co = pd.merge(combined_data_co, combined_data_diff_ns)

# 2024년 한 해 동안의 데이터 필터링
total_data_ns = combined_data_diff_ns[(combined_data_diff_ns['date'] >= '2024-01-01') & 
                                (combined_data_diff_ns['date'] <= '2024-12-31')]

year_data_ns = total_data_ns[(total_data_ns['date'] >= '2024-01-01') & 
                       (total_data_ns['date'] <= '2024-06-30')]

year_data_ns
# NO2 및 SO2의 카테고리가 같은 비율 계산
same_no2_category_count = year_data[year_data_ns['NO2_Category_Difference'] == 0].shape[0]
same_so2_category_count = year_data[year_data_ns['SO2_Category_Difference'] == 0].shape[0]

percent_same_no2_category = (same_no2_category_count / total_days) * 100
percent_same_so2_category = (same_so2_category_count / total_days) * 100

# 결과 출력
print(f"\n\n2024년 동안 NO2의 예측 카테고리가 실제와 같은 비율: {percent_same_no2_category:.2f}%")
print(f"2024년 동안 SO2의 예측 카테고리가 실제와 같은 비율: {percent_same_so2_category:.2f}%\n")

# 특정 날짜의 AQI 데이터를 출력 함수
def print_aqi_for_date_ns(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data_co[combined_data_co['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터")
        print(f"예상 NO2 : {round(row['Predicted_NO2'].values[0])} (Category: {row['NO2_Predict_Category'].values[0]})")
        print(f"예상 SO2 : {round(row['Predicted_SO2'].values[0])} (Category: {row['SO2_Predict_Category'].values[0]})")
        actual_no2 = round(row['Actual_NO2'].values[0]) if not np.isnan(row['Actual_NO2'].values[0]) else 'N/A'
        actual_so2 = round(row['Actual_SO2'].values[0]) if not np.isnan(row['Actual_SO2'].values[0]) else 'N/A'
        actual_no2_category = row['NO2_Actual_Category'].values[0] if not np.isnan(row['Actual_NO2'].values[0]) else 'N/A'
        actual_so2_category = row['SO2_Actual_Category'].values[0] if not np.isnan(row['Actual_SO2'].values[0]) else 'N/A'
        print(f"실제 NO2 : {actual_no2} (Category: {actual_no2_category})")
        print(f"실제 SO2 : {actual_so2} (Category: {actual_so2_category})")

# 특정 날짜의 AQI 데이터를 출력
print_aqi_for_date_ns(target_date)

2024년 미세먼지 예측 적중률
2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: 79.67%
2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: 98.90%

2024-12-20 대기질 예측 데이터
예상 PM25 : 91 (Category: 1)
예상 PM10 : 48 (Category: 1)

PM25 예측 모델의 결정 계수 (R-squared): 0.0393
PM10 예측 모델의 결정 계수 (R-squared): 0.0562
PM2.5 예측 모델의 RMSE: 34.2497
PM10 예측 모델의 RMSE: 21.5085

PM25 예측 모델의 카테고리별 RMSE: 2.1110
PM10 예측 모델의 카테고리별 RMSE: 2.0934
PM25 예측 모델의 카테고리별 결정 계수 (필터링 후): 0.7989
PM10 예측 모델의 카테고리별 결정 계수 (필터링 후): 0.9841


2024년 동안 CO의 예측 카테고리가 실제와 같은 비율: 100.00%
2024년 동안 O3의 예측 카테고리가 실제와 같은 비율: 97.80%

2024-12-20 대기질 예측 데이터
예상 CO : 5 (Category: 1)
예상 O3 : 35 (Category: 1)
실제 CO : N/A (Category: N/A)
실제 O3 : N/A (Category: N/A)


2024년 동안 NO2의 예측 카테고리가 실제와 같은 비율: 100.00%
2024년 동안 SO2의 예측 카테고리가 실제와 같은 비율: 100.00%

2024-12-20 대기질 예측 데이터
예상 NO2 : 27 (Category: 1)
예상 SO2 : 4 (Category: 1)
실제 NO2 : N/A (Category: N/A)
실제 SO2 : N/A (Category: N/A)


In [133]:

actual_seoul_ns

,date,Actual_NO2,Actual_SO2
0,2024-07-01,14.0,3.0
1,2024-07-02,10.0,3.0
2,2024-07-03,11.0,3.0
3,2024-07-04,6.0,3.0
4,2024-07-05,13.0,3.0
...,...,...,...
184,2024-03-27,30.0,4.0
185,2024-03-28,23.0,3.0
186,2024-03-29,25.0,3.0
187,2024-03-30,16.0,4.0


In [140]:
beijing_numeric.describe()


,pm25,pm10,o3,no2,so2,co
count,1912.000000,1912.000000,1912.000000,1912.000000,1912.000000,1912.0
mean,67.390608,36.593176,37.651969,6.635838,2.611557,0.0
std,30.903185,23.775680,13.050966,3.596224,1.222926,0.0
min,13.000000,6.000000,12.000000,1.000000,1.000000,0.0
25%,47.000000,21.000000,28.000000,4.000000,2.000000,0.0
50%,61.000000,32.000000,36.000000,6.000000,2.000000,0.0
75%,79.000000,46.000000,44.000000,8.000000,3.000000,0.0
max,192.000000,418.000000,109.000000,32.000000,11.000000,0.0


In [137]:
import pandas as pd
cities = pd.read_csv('전국태마파크링크추가.csv')['지역']
for i in list:
    print(i)

서울 서초구 잠원동
함안 법수면
부산 해운대구 중동
영덕 축산면
횡성 둔내면
김해 신문동
여주 산북면
당진 신평면
대구 달성군 가창면
경산 점촌동
천안 동남구 성남면
서울 광진구 구의동
춘천 사북면
김포 대곶면
거제 일운면
양평 옥천면
전북 완주군 구이면
인천 동구 송현동
용인 처인구 포곡읍
서울 종로구 옥인동
서울 서대문구 창천동
강릉 주문진읍
서울 송파구 문정동
양주 장흥면
용인 처인구 포곡읍
과천 막계동
고양 덕양구 효자동
부산 해운대구 우동
충주 목벌동
포천 소흘읍
경주 천군동
경주 천군동
논산 벌곡면
여수 소호동
경주 인왕동
천안 서북구 직산읍
아산 도고면
가평 가평읍
서울 영등포구 영등포동4가
부산 해운대구 중동
시흥 정왕동
서울 송파구 신천동
여수 만흥동
속초 설악동
경주 천군동
제주 서귀포시 성산읍
서울 영등포구 여의도동
포항 북구 죽장면
제주 제주시 해안동
고양 일산서구 대화동
울산 남구 옥동
제주 제주시 회천동
제주 제주시 한림읍
동해 삼화동
서울 영등포구 문래동3가
횡성 횡성읍
평택 진위면
화성 송동
제주 서귀포시 안덕면
서울 광진구 능동
제천 한수면
파주 문산읍
인천 중구 운서동
이천 모가면
서울 서초구 잠원동
부산 동래구 온천동
밀양 산내면
고양 덕양구 동산동
과천 막계동
속초 장사동


In [173]:

# 각 도시에서 시 또는 구까지만 추출하는 함수
def extract_city_name(city):
    if city.startswith("서울 ") or city.startswith("부산") or city.startswith("제주"):
        return city.split(" ")[1]
    elif city.startswith("전북 "):
        return '완주'
    else:
        return city.split(" ")[0]

# 각 도시에서 도시 이름만 추출하여 새로운 리스트 생성
city_names = [extract_city_name(city) for city in cities]

print("각 도시의 도시 이름:")


city_names = pd.Series(city_names).unique()
print(city_names)


각 도시의 도시 이름:
['서초구' '함안' '해운대구' '영덕' '횡성' '김해' '여주' '당진' '대구' '경산' '천안' '광진구' '춘천' '김포'
 '거제' '양평' '완주' '인천' '용인' '종로구' '서대문구' '강릉' '송파구' '양주' '과천' '고양' '충주' '포천'
 '경주' '논산' '여수' '아산' '가평' '영등포구' '시흥' '속초' '서귀포시' '포항' '제주시' '울산' '동해'
 '평택' '화성' '제천' '파주' '이천' '동래구' '밀양']


['D:\\FILE\\csv\\aju-dong,-geoje-si, gyeongnam-air-quality 거제.csv',
 'D:\\FILE\\csv\\bijeon-dong,-pyeongtaek-si, gyeonggi-air-quality 평택.csv',
 'D:\\FILE\\csv\\changjeon-dong,-icheon-si, gyeonggi-air-quality 이천.csv',
 'D:\\FILE\\csv\\cheongok-dong,-donghae-si, gangwon-air-quality 동해.csv',
 'D:\\FILE\\csv\\dangjinsicheongsa,-chungnam-air-quality 당진.csv',
 'D:\\FILE\\csv\\donghong-dong,-seogwipo-si, jeju-air-quality 서귀포.csv',
 'D:\\FILE\\csv\\gapyeong-eup,-gapyeong-gun, gyeonggi-air-quality 가평.csv',
 'D:\\FILE\\csv\\gaya-eup 함안.csv',
 'D:\\FILE\\csv\\geumchon-dong,-paju-si, gyeonggi-air-quality 파주.csv',
 'D:\\FILE\\csv\\gimhae-air-quality 김해.csv',
 'D:\\FILE\\csv\\gimnyangjang-dong,-yongin-si, gyeonggi-air-quality 용인.csv',
 'D:\\FILE\\csv\\gui-myeon,-jeonbuk-air-quality 완주.csv',
 'D:\\FILE\\csv\\gwacheon-dong,-gwacheon-si, gyeonggi-air-quality 과천.csv',
 'D:\\FILE\\csv\\gwangjin-gu,-seoul-air-quality 광진구.csv',
 'D:\\FILE\\csv\\haeun-daegu 해운대구.csv',
 'D:\\FILE\\csv\\hoam-dong,-chungju-si,

In [10]:
import pandas as pd

# CSV 파일 경로 설정
file_path = r'D:\FILE\csv\aju-dong,-geoje-si, gyeongnam-air-quality 거제.csv'

# CSV 파일을 데이터프레임으로 불러오기
df = pd.read_csv(file_path)

# 가장 최근 데이터(2024-07-08) 제거
df = df[df['date'] != '2024-07-08']

# 파일명에서 한글 ' 거제 ' 추가
file_name = file_path.split('\\')[-1].split('.')[0]
new_column_name = file_name.split(',')[-1].strip()
df['location'] = new_column_name

# 결과 확인
df.head()

,date,pm25,pm10,o3,no2,so2,co,location
0,2024/7/1,24,6,21,3,2,1,gyeongnam-air-quality 거제
1,2024/7/2,16,10,35,1,2,1,gyeongnam-air-quality 거제
2,2024/7/3,20,10,44,1,2,2,gyeongnam-air-quality 거제
3,2024/7/4,24,6,33,1,2,1,gyeongnam-air-quality 거제
4,2024/7/5,15,9,30,1,2,1,gyeongnam-air-quality 거제


In [24]:
import glob
import pandas as pd
from sqlalchemy import create_engine
import cx_Oracle

# 파일 경로 설정
directory_path = r'D:\FILE\csv'

# 해당 경로에 있는 모든 파일명 가져오기
file_names = glob.glob(directory_path + '/*')

# Oracle 데이터베이스 연결 정보
db_info = {
    'username': 'mis',
    'password': '12345',
    'host': 'localhost',
    'port': '1521',
    'service_name': 'xe'
}

# SQLAlchemy 엔진 생성
engine_str = f"oracle+cx_oracle://{db_info['username']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['service_name']}"
engine = create_engine(engine_str)

# 빈 데이터프레임 생성
df_list = []
df = pd.DataFrame()
# 파일 경로 리스트를 순회하며 데이터프레임을 리스트에 추가
for file_path in file_names:
    # CSV 파일을 데이터프레임으로 불러오기
    temp_df = pd.read_csv(file_path)

    # 파일명에서 첫 번째 공백(' ')을 기준으로 첫 번째 단어를 추출하여 'city' 컬럼에 추가
    file_name = file_path.split('\\')[-1].split('.')[0]
    first_word = file_name.split(' ')[-1]
    temp_df['city'] = first_word

    # 컬럼 이름에서 공백 제거
    temp_df.columns = temp_df.columns.str.replace(' ', '')

    # 'date' 컬럼을 'kdate'로 변경
    temp_df.rename(columns={'date': 'kdate'}, inplace=True)
    # 데이터프레임을 Oracle 데이터베이스에 저장
    table_name = 'kpred'
    temp_df.to_sql(table_name, engine, if_exists='append', index=False)
print("데이터프레임이 Oracle 데이터베이스의 kpred 테이블에 성공적으로 저장되었습니다.")

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [20]:
temp_df

,kdate,pm25,pm10,o3,no2,so2,co,city
0,2024/7/1,55,17,30,15,2,3,영등포구
1,2024/7/2,45,21,53,7,2,3,영등포구
2,2024/7/3,53,32,77,9,2,4,영등포구
3,2024/7/4,94,15,50,5,2,3,영등포구
4,2024/7/5,37,24,43,14,3,3,영등포구
...,...,...,...,...,...,...,...,...
3751,2014/3/29,,44,48,16,6,5,영등포구
3752,2014/3/30,,32,31,43,8,6,영등포구
3753,2014/3/31,,52,27,57,8,7,영등포구
3754,2014/4/27,,,21,20,5,5,영등포구


In [158]:

# Oracle 데이터베이스 연결 정보
db_info = {
    'username': 'mis',
    'password': '12345',
    'host': 'localhost',
    'port': '1521',
    'service_name': 'xe'
}

# SQLAlchemy 엔진 생성
engine_str = f"oracle+cx_oracle://{db_info['username']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['service_name']}"
engine = create_engine(engine_str)

# 빈 데이터프레임 생성
df_list = []

# 파일 경로 리스트를 순회하며 데이터프레임을 리스트에 추가
for file_path in file_names:
    # CSV 파일을 데이터프레임으로 불러오기
    temp_df = pd.read_csv(file_path)

    # 파일명에서 첫 번째 공백(' ')을 기준으로 첫 번째 단어를 추출하여 'city' 컬럼에 추가
    file_name = file_path.split('\\')[-1].split('.')[0]
    first_word = file_name.split(' ')[-1]
    temp_df['city'] = first_word

    # 컬럼 이름에서 공백 제거
    temp_df.columns = temp_df.columns.str.replace(' ', '')

    # 'date' 컬럼을 'kdate'로 변경
    temp_df.rename(columns={'date': 'kdate'}, inplace=True)

    # 데이터프레임을 Oracle 데이터베이스에 저장
    table_name = 'kpred'
    temp_df.to_sql(table_name, engine, if_exists='append', index=False)
print("데이터프레임이 Oracle 데이터베이스의 kpred 테이블에 성공적으로 저장되었습니다.")
temp_df


데이터프레임이 Oracle 데이터베이스의 kpred 테이블에 성공적으로 저장되었습니다.


,kdate,pm25,pm10,o3,no2,so2,co,city
0,2024/7/1,55,17,30,15,2,3,영등포구
1,2024/7/2,45,21,53,7,2,3,영등포구
2,2024/7/3,53,32,77,9,2,4,영등포구
3,2024/7/4,94,15,50,5,2,3,영등포구
4,2024/7/5,37,24,43,14,3,3,영등포구
...,...,...,...,...,...,...,...,...
3751,2014/3/29,,44,48,16,6,5,영등포구
3752,2014/3/30,,32,31,43,8,6,영등포구
3753,2014/3/31,,52,27,57,8,7,영등포구
3754,2014/4/27,,,21,20,5,5,영등포구


In [16]:
import cx_Oracle

# Oracle 데이터베이스 연결 정보
db_info = {
    'username': 'mis',
    'password': '12345',
    'dsn': 'localhost:1521/xe'  # 데이터베이스 접속 정보
}

# Oracle 데이터베이스에 연결
conn = cx_Oracle.connect(db_info['username'], db_info['password'], db_info['dsn'])

# 커서 생성
cursor = conn.cursor()

# SQL 쿼리 정의
sql = """
    DELETE FROM kpreddata
    WHERE TO_CHAR("date", 'YYYY/MM/DD') = '2024/07/10'
"""

try:
    # 쿼리 실행
    cursor.execute(sql)
    conn.commit()  # 변경 사항 저장

    print("데이터 삭제가 성공적으로 수행되었습니다.")

except cx_Oracle.DatabaseError as e:
    error, = e.args
    print("오류 발생:", error.message)

finally:
    # 연결 종료
    cursor.close()
    conn.close()


데이터 삭제가 성공적으로 수행되었습니다.


In [90]:
def extract_desired_part(file_path):
    # 파일 경로에서 파일 이름만 추출
    file_name = file_path.split('\\')[-1]
    
    # 파일 이름에서 확장자 제거
    file_name_without_extension = file_name.split('.')[0]
    
    # '-'로 분리하여 원하는 부분 선택 (처음 세 부분)
    parts = file_name_without_extension.split('-')
    desired_part = '-'.join(parts[:3])
    
    # 마지막 세 글자 제거
    desired_part = desired_part[:-3]
    
    return desired_part

# 예제 파일 경로

# 함수 호출
fnames = []
for file_path in file_names:
    result = extract_desired_part(file_path)
    fnames.append(result)

In [171]:
import requests

codes = range(1666, 1811)
data_list = []
codd = []
for code in codes:
    url = 'https://api.waqi.info/feed/@' + str(code) + '/?token=fc780f59a6e2c69acc41169b1a809665b9cd9477'
    res = requests.get(url)
    data = res.json()
    data_list.append(data['data']['city']['name']+ ':' +str(code))
 
for code in data_list:
    print(code)


Central, Singapore:1666
Gangnam-gu, Seoul, South Korea (강남구 서울):1667
Gangdong-gu, Seoul, South Korea (강동구 서울):1668
Gangbuk-gu, Seoul, South Korea (강북구 서울):1669
Gangseo-gu, Seoul, South Korea (강서구 서울):1670
Gwanak-gu, Seoul, South Korea (관악구 서울):1671
Gwangjin-gu, Seoul, South Korea (광진구 서울):1672
Guro-gu, Seoul, South Korea (구로구 서울):1673
Geumcheon-gu, Seoul, South Korea (금천구 서울):1674
Nowon-gu, Seoul, South Korea (노원구 서울):1675
Dobong-gu, Seoul, South Korea (도봉구 서울):1676
Dongdaemun-gu, Seoul, South Korea (동대문구 서울):1677
Dongjak-gu, Seoul, South Korea (동작구 서울):1678
Mapo-gu, Seoul, South Korea (마포구 서울):1679
Seodaemun-gu, Seoul, South Korea (서대문구 서울):1680
Seocho-gu, Seoul, South Korea (서초구 서울):1681
Seongdong-gu, Seoul, South Korea (성동구 서울):1682
Seongbuk-gu, Seoul, South Korea (성북구 서울):1683
Songpa-gu, Seoul, South Korea (송파구 서울):1684
Yangcheon-gu, Seoul, South Korea (양천구 서울):1685
Yeongdeungpo-gu, Seoul, South Korea (영등포구 서울):1686
Yongsan-gu, Seoul, South Korea (용산구 서울):1687
Bulgwang-dong, Eunpye

In [170]:
for code in data_list:
    print(code)

Central, Singapore:1666
Gangnam-gu, Seoul, South Korea (강남구 서울):1667
Gangdong-gu, Seoul, South Korea (강동구 서울):1668
Gangbuk-gu, Seoul, South Korea (강북구 서울):1669
Gangseo-gu, Seoul, South Korea (강서구 서울):1670
Gwanak-gu, Seoul, South Korea (관악구 서울):1671
Gwangjin-gu, Seoul, South Korea (광진구 서울):1672
Guro-gu, Seoul, South Korea (구로구 서울):1673
Geumcheon-gu, Seoul, South Korea (금천구 서울):1674
Nowon-gu, Seoul, South Korea (노원구 서울):1675
Dobong-gu, Seoul, South Korea (도봉구 서울):1676
Dongdaemun-gu, Seoul, South Korea (동대문구 서울):1677
Dongjak-gu, Seoul, South Korea (동작구 서울):1678
Mapo-gu, Seoul, South Korea (마포구 서울):1679
Seodaemun-gu, Seoul, South Korea (서대문구 서울):1680
Seocho-gu, Seoul, South Korea (서초구 서울):1681
Seongdong-gu, Seoul, South Korea (성동구 서울):1682
Seongbuk-gu, Seoul, South Korea (성북구 서울):1683
Songpa-gu, Seoul, South Korea (송파구 서울):1684


In [ ]:
data_list

In [164]:
import requests

res = requests.get(url)
data = res.json()
data['data']['city']['name']

'Seongnam-dong, Dong-gu, Daejeon, South Korea (성남동1 대전)'

In [ ]:
def extract_desired_part(file_path):
    # 파일 경로에서 파일 이름만 추출
    file_name = file_path.split('\\')[-1]
    
    # 파일 이름에서 확장자 제거
    desired_part = file_name.split('.')[0]
    
    
    
    return desired_part

# 예제 파일 경로

# 함수 호출
for file_path in file_names:
    result = extract_desired_part(file_path)
    print(result)  # 출력: aju-dong,-geoje-si, gyeongnam-air-quality

In [174]:
location_code_list = [
    ('거제', 1782), ('평택', 1718), ('이천', 1716), ('동해', 1732), ('당진', 1736), ('서귀포', 1805), ('가평', 1692), ('함안', 1783),
    ('파주', 1717), ('울산', 1762), ('용인', 1714), ('완주', 1799), ('과천', 1694), ('광진구', 1672), ('대구', 1768), ('충주', 1748),
    ('횡성', 1810), ('화성', 1721), ('제주', 1802), ('포항', 1781), ('밀양', 1763), ('논산', 1735), ('강릉', 1731), ('양주', 1709),
    ('제천', 1745), ('경주', 1777), ('천안', 1739), ('서울', 1679), ('고양', 1693), ('속초', 1804), ('인천', 1726), ('송파구', 1684),
    ('김포', 1706), ('양평', 1710), ('여수', 1792), ('여주', 1711), ('영덕', 1808),
    ('서초구', 1681), ('해운대구', 1811), ('김해', 5526), ('경산', 1776), ('춘천', 1692), ('종로구', 1689), ('서대문구', 1680), ('포천', 1719),
    ('아산', 1738), ('영등포구', 1686), ('시흥', 1705), ('동래구', 1754)
]
len(location_code_list)

49

In [2]:
import requests
import pandas as pd
from datetime import date
from sqlalchemy import create_engine

# AQICN 사이트에서 크롤링 하는 함수
# Oracle 데이터베이스 연결 정보
db_info = {
    'username': 'mis',
    'password': '12345',
    'host': 'localhost',
    'port': '1521',
    'service_name': 'xe'
}
# SQLAlchemy 엔진 생성
engine_str = f"oracle+cx_oracle://{db_info['username']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['service_name']}"
engine = create_engine(engine_str)
# 주어진 위치와 코드 매핑 딕셔너리
location_code_list = [
    ('거제', 1782), ('평택', 1718), ('이천', 1716), ('동해', 1732), ('당진', 1736), ('서귀포', 1805), ('가평', 1692), ('함안', 1783),
    ('파주', 1717), ('울산', 1762), ('용인', 1714), ('완주', 1799), ('과천', 1694), ('광진구', 1672), ('대구', 1768), ('충주', 1748),
    ('횡성', 1810), ('화성', 1721), ('제주', 1802), ('포항', 1781), ('밀양', 1763), ('논산', 1735), ('강릉', 1731), ('양주', 1709),
    ('제천', 1745), ('경주', 1777), ('천안', 1739), ('서울', 1679), ('고양', 1693), ('속초', 1804), ('인천', 1726), ('송파구', 1684),
    ('김포', 1706), ('양평', 1710), ('여수', 1792), ('여주', 1711), ('영덕', 1808),
    ('서초구', 1681), ('해운대구', 1811), ('김해', 5526), ('경산', 1776), ('춘천', 1692), ('종로구', 1689), ('서대문구', 1680), ('포천', 1719),
    ('아산', 1738), ('영등포구', 1686), ('시흥', 1705), ('동래구', 1754), ('랴오청', 1519)
]
data_list = []
# 각 위치 코드에 대해 대기질 정보를 가져오기
for i in range(len(location_code_list)):
    # 첫 번째 튜플(city 정보) 가져오기
    first_tuple = location_code_list[i]
    city = first_tuple[0]
    code = first_tuple[1]
    url = 'https://api.waqi.info/feed/@' + str(code) + '/?token=fc780f59a6e2c69acc41169b1a809665b9cd9477'
    res = requests.get(url)
    data = res.json()
    data_list.append(data)
    spm10 = data_list[i]['data']['iaqi']['pm10']['v']
    spm25 = data_list[i]['data']['iaqi']['pm25']['v']
    so3 = data_list[i]['data']['iaqi']['o3']['v']
    sno2 = data_list[i]['data']['iaqi']['no2']['v']
    sso2 = data_list[i]['data']['iaqi']['so2']['v']
    sco = data_list[i]['data']['iaqi']['co']['v']
    # 오늘의 날짜 구하기
    kdate = date.today().strftime("%Y-%m-%d")
    # 데이터프레임 생성
    data = {
        'kdate': [kdate],
        'pm10': [spm10],
        'pm25': [spm25],
        'o3': [so3],
        'no2': [sno2],
        'so2': [sso2],
        'co': [sco],
        'city': [city]
    }
    df = pd.DataFrame(data)
    # 데이터프레임을 Oracle 데이터베이스에 저장
    table_name = 'kpred'
    df.to_sql(table_name, engine, if_exists='append', index=False)


In [4]:
# AQICN 사이트에서 크롤링 하는 함수
# Oracle 데이터베이스 연결 정보
db_info = {
    'username': 'mis',
    'password': '12345',
    'host': 'localhost',
    'port': '1521',
    'service_name': 'xe'
}
# SQLAlchemy 엔진 생성
engine_str = f"oracle+cx_oracle://{db_info['username']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['service_name']}"
engine = create_engine(engine_str)

sqldf = pd.read_sql("select * from weather_forecast", engine).head(100)
sqldf

,code,name,fdate,temp_min,temp_max,temp_avg,rain_avg,wind_avg,wind_deg,wind_max,hum_avg,rainp_am,rainp_pm
0,1132052100,도봉구,2024-07-13,21,32,27,0,1,북동풍,2,62,10%,10%
1,1123060000,동대문구,2024-07-11,23,32,27,0,1,남동풍,3,69,0%,80%
2,1123060000,동대문구,2024-07-12,23,32,27,0,1,남동풍,2,67,10%,10%
3,1123060000,동대문구,2024-07-13,23,33,28,0,1,북동풍,2,60,20%,20%
4,1159051000,동작구,2024-07-11,22,32,26,0,1,남동풍,3,73,0%,70%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1120059000,성동구,2024-07-12,22,33,27,0,1,남동풍,2,67,20%,60%
96,1120059000,성동구,2024-07-13,23,33,28,0,1,북동풍,2,60,20%,10%
97,1129066000,성복구,2024-07-11,23,32,27,0,1,남동풍,3,69,0%,80%
98,1129066000,성복구,2024-07-12,23,32,27,0,1,남동풍,2,67,10%,10%


In [123]:
spm10 = data_list[0]['data']['iaqi']['pm10']['v']
spm25 = data_list[0]['data']['iaqi']['pm25']['v']
so3 = data_list[0]['data']['iaqi']['o3']['v']
sno2 = data_list[0]['data']['iaqi']['no2']['v']
sso2 = data_list[0]['data']['iaqi']['so2']['v']
sco = data_list[0]['data']['iaqi']['co']['v']
print(spm10, spm25, so3, sno2, sso2, sco)
first_tuple = location_code_list[0]
value_of_first_tuple = first_tuple[1]

13 42 26.8 2.3 3 1.8


In [122]:
location_code_list = [
    ('거제', 1782), ('평택', 1718), ('이천', 1716), ('동해', 1732), ('당진', 1736), ('서귀포', 1805), ('가평', 1692), ('함안', 1783),
    ('파주', 1717), ('울산', 1762), ('용인', 1714), ('완주', 1799), ('과천', 1694), ('광진구', 1672), ('대구', 1768), ('충주', 1748),
    ('횡성', 1810), ('화성', 1721), ('제주', 1802), ('포항', 1781), ('밀양', 1763), ('논산', 1735), ('강릉', 1731), ('양주', 1709),
    ('제천', 1745), ('경주', 1777), ('천안', 1739), ('서울', 1679), ('고양', 1693), ('속초', 1804), ('인천', 1726), ('송파구', 1684),
    ('김포', 1706), ('양평', 1710), ('여수', 1792), ('여주', 1711), ('영덕', 1808)
]

first_tuple = location_code_list[0]
value_of_first_tuple = first_tuple[1]

print(value_of_first_tuple)  # Output: 1782

1782


In [125]:
import pandas as pd
from datetime import date
# 오늘의 날짜 구하기
kdate = date.today().strftime("%Y-%m-%d")

# 첫 번째 튜플(city 정보) 가져오기
first_tuple = location_code_list[0]
city = first_tuple[0]

# 데이터프레임 생성
data = {
    'kdate': [kdate],
    'pm10': [spm10],
    'pm25': [spm25],
    'o3': [so3],
    'no2': [sno2],
    'so2': [sso2],
    'co': [sco],
    'city': [city]
}

df = pd.DataFrame(data)

# 데이터프레임 출력
df

,kdate,pm10,pm25,o3,no2,so2,co,city
0,2024-07-09,13,42,26.8,2.3,3,1.8,거제
